Why
- Look for issues in the data, prelude to data cleaning
- Understand the data
- Communicate your findings

Prerequisites
-  Gaussian distribution, uniform distribution
-  Correlation
-  Logarithm, log-scale
-  Matplotlib
-  Linear algebra (eigenvalues)

Univariate, numeric
- Numbers: mean, median, sd, IQR, skew, kurt, quantiles, missing, mode, min, max
- zero inflation, magic numbers
- boxplot, histogram, density, log-scale, fitted distribution
- comparison of the tails, fit the tails, quantile-quantile plot
- outliers

Univariate, qualitative
- Numbers: number of unique values (constant/id)
- barplot (counts)

Time series
- Univariate EDA
- Autocorrelation
- Stationarity
- Seasonality, trend

Bivariate
- Univariate EDA for each variable
- Numbers: univariate ones + correlation; missingness patterns? anscombe / datasaurus
- Scatterplot, hexbin (+log), regression line (+interval), lowess
- Clustering (k-means, hdbscan)
- density estimation, copula, median per quantile

Bivariate, with one qualitative variable

Multivariate
- Univariate EDA, in particular the missing values
- pairs plot, corrplot (+order from hclust)
- hclust for the variables
- dimension reduction: PCA, t-SNE, UMAP (+clustering)

Panel data
- Missing values: number, distribution

Text

Images

Graphs

Matrices

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os, re, math, datetime, logging, warnings
from collections import Counter
import scipy, sklearn
from umap import UMAP

import sklearn.datasets
from pydataset import data  # Many datasets from R
from palmerpenguins import load_penguins
from gapminder import gapminder
from pandas_datareader import wb

import matplotlib.patheffects as path_effects
from matplotlib.gridspec import GridSpec
from matplotlib.patches import FancyArrowPatch, Circle, Ellipse
import sklearn
import sklearn.cluster
import statsmodels
import statsmodels.graphics.mosaicplot
import statsmodels.tsa.stattools
import statsmodels.nonparametric.kernel_density
import torch
import random
import yfinance as yf
import matplotlib
import dateparser
import pandas_datareader
import cvxpy as cp
import squarify
import dateparser
from itertools import cycle, islice
from io import StringIO
from KDEpy import FFTKDE
from scipy.cluster.hierarchy import ward, leaves_list
from scipy.spatial.distance import squareform, pdist

In [ ]:
def cov2cor(V):
    """
    Convert a covariance matrix to a correlation matrix by dividing each row and each column
    by the standard deviation (square root of the corresponding diagonal element).

    Input: V: Pandas DataFrame, containing a square, symmetric matrix, with matching row and column names
              May also work with a square matrix
              We do not check if the matrix is positive definite, but the diagonal elements should be positive
    Output: Pandas dataframe, of the same size, with the same row and column names, containing the corresponding correlation matrix.

    Example:
        import numpy as np
        import pandas as pd
        X = np.random.normal( size=(100,5) )
        X = pd.DataFrame(X)
        V = X.cov()
        C = cov2cor(V)
    """
    D = 1 / np.sqrt( np.diag(V) )
    k = D.shape[0]
    D = D * np.ones( shape = (k,k) )
    C = D * V * D.T
    return C

def stackplot(
    x, y = None,
    N      = 11,
    ax     = None,
    legend = False,
    labels = [],
    loc    = 'best',
    colors = None,
    first  = None,
    **kwargs
):
    """
    Stacked area plot, allowing for negative values.

    Inputs: x, y:   Data to plot
            N:      Number of interpolation points (we need to add more points to account for sign changes)
            ax:     Where to put the plot
            legend: Boolean, whether to add a legend
            labels: Labels for the legend, if they cannot be inferred from the data
            loc:    Position of the legend, e.g., 'best' or (1.04,0) (give an incorrect string to see the accepted values)
            colors: List of colours to use; defaults to plt.get_cmap("tab10").colors
            first:  Number of categories to include in the legend (if there are too many of them for the legend to be helpful);
                    if the colors are not provided, use shades of gray for the categories not in the legend
            ...:    Passed to ax.stackplot()
    Output: None

    Example:
        from vz import stackplot
        import numpy as np
        import pandas as pd
        x = np.random.normal(size=(10,4))
        x = pd.DataFrame(x, columns = np.linspace(0,1,4))
        x = np.cumsum(x)
        stackplot(x, legend=True)
    """
    ## To smooth out the sign changes, add intermediate points
    ## (There is only a problem when the sign of one of the time series changes,
    ## and it can be fixed by adding the point (x-value) where the sign changes,
    ## i.e., when the value becomes zero. It is easier (and also more scalable,
    ## if there are hundreds of time series) to add more intermediate points:
    ## if they are sufficiently dense, everything will look fine.)

    if y is None:
        y = x.values
        if len(labels) == 0:
            labels = x.index
        x = x.columns
    else:
        assert (len(x.shape) == 1) and (len(y.shape) == 2), "Did you swicth the x and y arguments?"
        assert len(x) == y.shape[1], f"Inconsistent shapes: len(x) is {len(x)}, y.shape is {y.shape}"

    if isinstance(y, pd.DataFrame):
        if len(labels) == 0:
            labels = y.index
        y = y.values
    if isinstance(x, pd.Series):
        x = x.values

    if first is not None:
        labels = [ v if k < first else '_no_label_' for k,v in enumerate(labels) ]

    if colors is None:
        colors = cycle( plt.get_cmap("tab10").colors )
        if first is not None:
            colors = chain(
                islice( cycle( colors ), 0, first ),
                cycle( [ 'lightgray', 'darkgray' ] )
            )
        colors = islice( colors, 0, len(labels) )
        colors = list( colors )

    assert not isinstance(x[0], str), f"The x-axis should be numeric (dates are fine, as well), got an array of {type(x[0])}: {x}"

    date_axis = isinstance(x[0], pd.Timestamp) or isinstance(x[0], np.datetime64)
    if date_axis:
        x = matplotlib.dates.date2num(x)
        x = 1.0 * x

    xs = np.stack( [ x[:-1], x[1:] ] )
    xs = np.apply_along_axis( lambda u: np.linspace(u[0],u[1],N+1)[:N], 0, xs ).T.flatten()
    ys = []
    for i in range(y.shape[0]):
        ys.append( scipy.interpolate.interp1d(x,y[i,:])(xs) )
    ys = np.array(ys)

    ## Plot separately the positive and negative values
    y_pos = np.where( ys >= 0, ys, 0 )
    y_neg = np.where( ys <= 0, ys, 0 )

    ## Plot
    ax_was_None = ax is None
    if ax_was_None:
        fig, ax = plt.subplots()
    ax.stackplot(xs, y_pos, labels = labels, colors = colors, **kwargs)
    ax.stackplot(xs, y_neg, labels = [],     colors = colors, **kwargs)
    ax.set_xlim( min(x), max(x) )
    if date_axis:
        dx = max(x) - min(x)
        if dx > 7000:
            axis_decade(ax)
        elif dx > 3500:
            axis_year(ax, '%y')
        elif dx > 600:
            axis_year(ax)
        else:
            axis_month(ax)
    if legend:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles[::-1], labels[::-1], loc=loc )
    if ax_was_None:
        plt.show()

def add_ellipse(
    V,
    mu,
    radius    = 1,
    ax        = None,
    color     = 'tab:blue',
    linewidth = 3,
):
    """
    Add an ellipse to a plot, from its center and covariance matrix

    Note:
    - Matplotlib will not automatically set xlim and ylim to make the ellipse visible:
      unless there is a cloud of points where you are drawing the ellipse,
      you will need to set them yourself.

    Inputs: V:         variance matrix
            mu:        center of the ellipse
            ax:        where to draw the ellipse
            color:     passed to Ellipse.set_color
            linewidth: passed to Ellipse.set_linewidth
    Output: the ellipse, should you want to modify it further

    Example:
        X = pd.DataFrame( np.random.normal( size=(100,2) ) )
        R = np.random.normal( size = (2,2) )
        X = X @ R
        V = X.cov()
        mu = X.mean()
        fig, ax = plt.subplots()
        ax.scatter( X.iloc[:,0], X.iloc[:,1],  )
        for radius in [1,2,3,4,5]:
            e = add_ellipse( V, mu, radius, ax=ax )
            e.set_color( colours_quintile[radius-1] )
            e.set_facecolor('none')
        fig.tight_layout()
        plt.show()
    """

    assert isinstance( V, pd.DataFrame ) or isinstance( V, np.ndarray )
    if isinstance( V, pd.DataFrame ):
        V = V.values
    assert V.shape == (2,2), f"Expecting a 2×2 matrix, but {V.shape=}"
    assert isinstance( mu, pd.Series ) or isinstance( mu, np.ndarray ) or isinstance( mu, tuple ) or isinstance( mu, list )
    if isinstance( mu, pd.Series ):
        mu = mu.values
    else:
        mu = np.array( mu )
    assert mu.shape == (2,), f"Expecting a 2-element vector, but {mu.shape=}"
    assert np.all( np.isfinite(mu) ), f"mu constains non-finite values:\n{mu}"
    assert np.all( np.isfinite(V ) ),  f"V constains non-finite values:\n{V}"

    D, U = np.linalg.eigh(V)
    D = np.clip(D, 0, np.inf)
    sigma = np.sqrt( D )
    theta = -np.arctan2(U[1,0],U[0,0])

    e = Ellipse(
        xy = mu,
        width  = radius * sigma[0],
        height = radius * sigma[1],
        angle = - np.rad2deg( theta ),
    )
    e.set_color(color)
    e.set_linewidth(linewidth)
    e.set_facecolor('none')
    if ax is not None:
        ax.add_artist(e)
    return e

def plot_copula(x,y, ax = None, title = None, unif=True, cmap='Blues', N=None, implementation = 'FFTKDE'):
    """
    Plot the copula density of (x,y).

    Inputs: x, y: Numpy vectors, of the same length
            ax: where to plot
            title: given to ax.set_title()
            unif: whether to uniformize x and y or gaussianize them
            cmap: colourmap to use
            N: number of points to use to estimate the density
               (I suspect the algorithm used by Python is quadratic -- the Fast Gaussian Transform should be faster)
            implementation: 'FFTKDE', 'scipy', 'statsmodels', 'sklearn'
    Output: None

    Examples:
        from vz import plot_copula
        import numpy as np
        import matplotlib.pyplot as plt

        x = np.random.normal( size=5000 )
        y = np.random.normal( size=5000 )
        plot_copula(x, y, title='Independence copula')
        plot_copula(x, x+y, title='Gaussian copula')

        from scipy.stats import multivariate_t
        rv = multivariate_t([1.0, -0.5], [[2.1, 0.3], [0.3, 1.5]], df=1)
        xy = rv.rvs(size=5000)
        x, y = xy[:,0], xy[:,1]
        plot_copula(x, y, title='Student copula')

        import copulae
        c = copulae.ClaytonCopula(dim=2, theta=1)
        xy = c.random(5000)
        x, y = xy[:,0], xy[:,1]
        plot_copula(x, y, title='Clayton copula')

        x = np.random.normal( size=5000 )
        y = np.random.normal( size=5000 )
        xy = rv.rvs(size=5000)
        fig, axs = plt.subplots(1,2, figsize=(8,4))
        plot_copula(x, y, unif=False, title='Gaussian', ax=axs[0])
        plot_copula(xy[:,0], xy[:,1], unif=False, title='Student', ax=axs[1])
        for ax in axs:
            ax.set_xlim(-2,2)
            ax.set_ylim(-2,2)
        fig.suptitle("With Gaussian margins")
        fig.tight_layout()
        plt.show()
    """
    assert implementation in ['FFTKDE', 'scipy', 'statsmodels', 'sklearn']
    i = np.isfinite(x) & np.isfinite(y)
    x = x[i]
    y = y[i]
    if N is not None and N < len(x):
        i = np.random.choice( len(x), N, replace = False )
        x = x[i]
        y = y[i]
    x = uniformize(x)
    y = uniformize(y)
    if unif:
        xmin, xmax, ymin, ymax = 0,1, 0,1
    else:
        x = scipy.stats.norm.ppf(x)
        y = scipy.stats.norm.ppf(y)
        xmin, xmax = min(x), max(x)
        ymin, ymax = min(y), max(y)
    ax_was_None = ax is None
    if ax_was_None:
        fig, ax = plt.subplots(figsize=(4,4))

    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])

    if implementation == 'scipy':
        # Scipy: acceptable until 10_000 points
        kernel = scipy.stats.gaussian_kde( np.vstack([x, y]) )
        f = np.reshape(kernel(positions).T, xx.shape)

    elif implementation == 'sklearn':
        # Sklearn: slower, and no automated bancwidth selection (unless you use GridSearchCV(kde,{'bandwidth':bandwidth}))
        kde = sklearn.neighbors.KernelDensity(bandwidth=.05).fit( np.stack([x,y]).T )
        log_density = kde.score_samples( positions.T )
        f = np.exp( np.reshape(log_density, xx.shape) )

    elif implementation == 'statsmodels':
        # Statsmodels: as slow as sklearn
        m = statsmodels.nonparametric.kernel_density.KDEMultivariate([x,y], 'cc')
        f = np.reshape( m.pdf(positions), xx.shape )

    else:
        # KDEpy: 20 times faster; no automated bandwidth selection in 2d; no idea how to specify the grid (the examples are in 1d)
        bw = statsmodels.nonparametric.kernel_density.KDEMultivariate([x,y], 'cc').bw.mean()
        k = 256
        u, f = FFTKDE(bw=bw).fit( np.stack([x,y]).T )((k,k))
        xx, yy = np.unique(u[:, 0]), np.unique(u[:, 1])
        f = f.reshape(k,k).T

    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.contourf(xx, yy, f, cmap=cmap)
    #ax.imshow(np.rot90(f), cmap='Blues', extent=[xmin, xmax, ymin, ymax])
    ax.contour(xx, yy, f, colors='k')
    #ax.clabel(cset, inline=1, fontsize=10)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    if title is not None:
        ax.set_title(title)
    if ax_was_None:
        plt.show()

def remove_empty_axes(axs: np.ndarray) -> None:
    """
    Remove empty subplots

    Inputs: axs: numpy array, returned by plt.subplots()
    Output: None

    Examples:
        import matplotlib.pyplot as plt
        import numpy as np
        fig, axs = plt.subplots(2,2)
        x = np.random.normal(size=20)
        y = np.random.normal(size=20)
        axs[0,0].scatter(x, y)
        axs[1,0].scatter(x, y)
        axs[0,1].scatter(x, y)
        remove_empty_axes(axs)
        fig.tight_layout()
        plt.show()
    """
    for ax in axs.flatten():
        if (not ax.lines) and (not ax.collections) and (not ax.has_data()):
            ax.axis('off')

def mfrow(
        n:      int,
        aspect: float = 29.7/21,
        width:  float = 29.7,
        height: float = 21,
        pages:  int   = 1
):
    """
    Compute a layout (number of rows and columns) to put n plots,
    as close as possible to the desired aspect ratio,
    with as few empty cells as possible,
    for the given plot dimensions.

    Also see: remove_empty_axes

    Inputs:  n: number of subplots
             aspect: desired aspect ratio of the subplots
             width: width of the (super)plot
             height: height of the (super)plot
             pages: number of (super)plots (untested -- I think it does not do what I want)
    Outputs: nr: Number of rows
             nc: Number of columns
    """
    best = (1,1)
    best_value = float('inf')
    for nc in range(1,n+1):
        nr = math.ceil( n / nc / pages ) * pages
        a = ( width / nc ) / ( height / nr )
        if abs( a - aspect ) < best_value:
            best_value = abs( a - aspect )
            best = (nr, nc)
    return best

def add_ellipse(
    V,
    mu,
    radius    = 1,
    ax        = None,
    color     = 'tab:blue',
    linewidth = 3,
):
    """
    Add an ellipse to a plot, from its center and covariance matrix

    Note:
    - Matplotlib will not automatically set xlim and ylim to make the ellipse visible:
      unless there is a cloud of points where you are drawing the ellipse,
      you will need to set them yourself.

    Inputs: V:         variance matrix
            mu:        center of the ellipse
            ax:        where to draw the ellipse
            color:     passed to Ellipse.set_color
            linewidth: passed to Ellipse.set_linewidth
    Output: the ellipse, should you want to modify it further

    Example:
        X = pd.DataFrame( np.random.normal( size=(100,2) ) )
        R = np.random.normal( size = (2,2) )
        X = X @ R
        V = X.cov()
        mu = X.mean()
        fig, ax = plt.subplots()
        ax.scatter( X.iloc[:,0], X.iloc[:,1],  )
        for radius in [1,2,3,4,5]:
            e = add_ellipse( V, mu, radius, ax=ax )
            e.set_color( colours_quintile[radius-1] )
            e.set_facecolor('none')
        fig.tight_layout()
        plt.show()
    """

    assert isinstance( V, pd.DataFrame ) or isinstance( V, np.ndarray )
    if isinstance( V, pd.DataFrame ):
        V = V.values
    assert V.shape == (2,2), f"Expecting a 2×2 matrix, but {V.shape=}"
    assert isinstance( mu, pd.Series ) or isinstance( mu, np.ndarray ) or isinstance( mu, tuple ) or isinstance( mu, list )
    if isinstance( mu, pd.Series ):
        mu = mu.values
    else:
        mu = np.array( mu )
    assert mu.shape == (2,), f"Expecting a 2-element vector, but {mu.shape=}"
    assert np.all( np.isfinite(mu) ), f"mu constains non-finite values:\n{mu}"
    assert np.all( np.isfinite(V ) ),  f"V constains non-finite values:\n{V}"

    D, U = np.linalg.eigh(V)
    D = np.clip(D, 0, np.inf)
    sigma = np.sqrt( D )
    theta = -np.arctan2(U[1,0],U[0,0])

    e = Ellipse(
        xy = mu,
        width  = radius * sigma[0],
        height = radius * sigma[1],
        angle = - np.rad2deg( theta ),
    )
    e.set_color(color)
    e.set_linewidth(linewidth)
    e.set_facecolor('none')
    if ax is not None:
        ax.add_artist(e)
    return e

def corrplot(
        C,
        ax      = None,
        vmin    = -1,
        vmax    = +1,
        cmap    = 'RdBu',
        title   = None,
        figsize = (12, 12),
        order   = False,
        aspect  = None,
        labels  = False,
        ticks   = True,
        interpolation = 'nearest',
):
    """
    Plot a correlation matrix.

    Positive correlations are in blue, negative correlations in red.
    The correlations can be written in white in the cells -- if they are not legible, they are not significant.
    The rows and columns can be reordered, using a hierarchical clustering.

    Inputs: C:  DataFrame, part of a correlation matrix
            ax: where to put the plot; use None to create a new plot
            vmin, vmax: minimum and maximum, for the colour gradient; you probably want 0 to be in the middle of this interval
            cmap: colourmap; use a divergent colourmap, with white (or grey) in the middle
            title: str
            figsize:
            order: boolean; whether to re-order the rows and the columns using hierarchical clustering
            aspect: None for square cells, 'auto' for rectangular ones
            label: boolean; whether to write the correlations in the cells (in white: if you cannot read them, they were not significant)
            interpolation: passed to imshow()
    Output: None

    Example:
        from vz import corrplot, LETTERS
        import numpy as np
        import pandas as pd
        x = np.random.normal(size=(100,11))
        x = pd.DataFrame( x, columns = LETTERS[:x.shape[1]] )
        C = x.corr()
        C = np.sign(C) * np.abs(C) ** (1/2)
        corrplot(C, figsize=(6,6), order=True, labels = True)
    """
    assert len(C.shape) == 2
    if not isinstance(C, pd.DataFrame):
        C = pd.DataFrame(C)
    if order:
        ## First, check and fix the correlation matrix
        C = C.copy()
        if np.any( ( C < -1 ) | ( C > +1 ) ):
            LOG( "Warning: the correlation matrix has values outside [-1,+1]" )
            C = np.clip(C, -1, 1)
        if np.any( ~ np.isfinite(C) ):
            LOG( "Warning: the correlation matrix contains infinite and/or missing values" )
            C[ ~ np.isfinite(C) ] = 0

        ## It can be a correlation matrix or just part of it
        if ( C.shape[0] == C.shape[1] ) and np.all( C.index == C.columns ):

            if np.any( np.diag(C) != 1 ):
                LOG( "Warning: the correlation matrix has values != 1 on the diagonal" )
                np.fill_diagonal(C.values, 1)

            #i = leaves_list( ward(pdist(C)) )            # Distances between the columns of the correlation matrix
            #i = leaves_list( ward(squareform(1-C) ) )    # (Squared) distance matrix from the correlation matrix
            i = leaves_list( ward(squareform( np.sqrt(1-C), checks=False )))
            C = C.iloc[i,i]

        else:
            i = leaves_list( ward(pdist(C)) )            # Distances between the columns of the correlation matrix
            j = leaves_list( ward(pdist(C.T)) )
            C = C.iloc[i,j]

    ax_was_None = ax is None
    if ax is None:
        fig, ax = plt.subplots( figsize = figsize )
    ax.imshow(C, vmin = vmin, vmax = vmax, cmap = cmap, aspect = aspect, interpolation = interpolation )

    if labels:
        for x in range(C.shape[1]):
            for y in range(C.shape[0]):
                ax.text( x, y, f"{C.iloc[y,x]:.2f}", ha='center', va='center', color='white' )

    if ticks:
        ax.set_xticks( range(C.shape[1]) )
        ax.set_xticklabels( C.columns, rotation = 90 )
        ax.set_yticks( range(C.shape[0]) )
        ax.set_yticklabels( C.index )
    else:
        ax.set_xticks([])
        ax.set_yticks([])

    if title is not None:
        ax.set_title(title)
    if ax_was_None:
        plt.show()

def remove_scientific_notation_from_vertical_axis(ax, deprecated_argument=None):
    """
    Remove the scientific notation from the vertical axis tick labels.
    If the scale is logarithmic but spans less than one or two orders of magnitude.
    """

    if deprecated_argument is None:
        fig = ax.get_figure()
    else:
        # The old version of this function was taking fig, ax as argument...
        # TODO: issue a deprecation warning
        fig, ax = ax, deprecated_argument

    fig.canvas.draw()

    def remove_scientific_notation(text = '$\\mathdefault{2\\times10^{-2}}$'):
        if text == '':
            return text
        expr = r'\$\\mathdefault\{((.*)\\times)?10\^\{(.*)\}\}\$'
        mantissa = re.sub( expr, r'\2', text )
        exponent = re.sub( expr, r'\3', text )
        if mantissa == '':
            mantissa = 1
        mantissa = float(mantissa)
        exponent = float(exponent)
        result = mantissa * 10 ** exponent
        return f'{float(f"{result:.4g}"):g}'

    labels = ax.yaxis.get_ticklabels()
    for label in labels:
        a = label.get_text()
        b = remove_scientific_notation(a)
        label.set_text(b)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', message = "FixedFormatter should only be used together with FixedLocator" )
        ax.yaxis.set_ticklabels(labels)

    labels = ax.yaxis.get_minorticklabels()
    for label in labels:
        a = label.get_text()
        b = remove_scientific_notation(a)
        label.set_text(b)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', message = "FixedFormatter should only be used together with FixedLocator" )
        ax.yaxis.set_ticklabels(labels, minor=True)

def axis_decade(ax):
    """
    On the horizontal axis, add small ticks every year, large ticks every decade, and decade labels.

    Inputs: ax: "axis", returned by plt.subplots()
    Output: None

    Example:
        import yfinance as yf
        import matplotlib.pyplot as plt
        x = yf.Ticker("^GSPC").history(period="max")['Close']
        fig, ax = plt.subplots( figsize=(8,4) )
        ax.plot(x)
        axis_decade(ax)
        axis.set_yscale("log")
        plt.show()
    """

    ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(10))
    ax.xaxis.set_minor_locator(matplotlib.dates.YearLocator(5,month=7, day=1))
    ax.xaxis.set_major_formatter(matplotlib.ticker.NullFormatter())

    def f(u):
        date = matplotlib.dates.num2date(u)
        if date.year % 10 == 5:
            s = date.strftime("%Y")
            s = re.sub( "5$", "0s", s )
            return s
        return ''
    ax.xaxis.set_minor_formatter( lambda u,v: f(u) )

    for tick in ax.xaxis.get_minor_ticks():
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
        tick.label1.set_horizontalalignment('center')

def axis_year(ax, format="%Y"):
    """
    On the horizontal axis, add small ticks every year, large ticks every decade, and year labels in each interval.

    Inputs: ax:     "axis", returned by plt.subplots()
            format: date format used for the year, e.g., "%Y" (4 digits) or "%y" (2 digits)
    Output: None

    Example:
        import yfinance as yf
        import matplotlib.pyplot as plt
        x = yf.Ticker("^GSPC").history(period="5y")['Close']
        fig, ax = plt.subplots()
        ax.plot(x)
        axis_year(ax)
        plt.show()
    """
    ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
    ax.xaxis.set_minor_locator(matplotlib.dates.YearLocator(month=7, day=1))
    ax.xaxis.set_major_formatter(matplotlib.ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter(format))
    for tick in ax.xaxis.get_minor_ticks():
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
        tick.label1.set_horizontalalignment('center')
    ticks = ax.xaxis.get_major_ticks()
    xlocs = ax.get_xticks()
    for pos, tick in zip(xlocs, ticks):
        date = matplotlib.dates.num2date(pos)
        if date.year % 10 == 0:
            tick.tick1line.set_markersize(15)

def pcolormesh(
        x, y = None, z = None,
        ax = None, **kwargs
):
    """
    Wrapper around Matplotlib's pcolormesh for the case
    when x and y have as many elements as z has columns and rows.

    Matplotlib's pcolormesh needs one more value in x and y than in z.
    Otherwise, it *silently* discards the last row and.or column of z.
    Instead, add the missing values:
    - For numeric values, or dates, use the midpoints between the provided points,
      and add a point at the begining and at the end, using half the distance to the nearest point.
    - For strings, use equal-spaced points.
    It also works if there is only one point.

    The code was only tested with Numpy arrays, not Pandas Series and DataFrames.

    Inputs: x:      Numpy array of length n
            y:      Numpy array of length m
            z:      Numpy array of shape (m,n)
            ax:     Where to add the plot
            kwargs: Passed to matplotlib's pcolormesh
    Output: None

    Example:
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        from vz import pcolormesh, axis_month
        y = [1,2,3,5,10]
        x = pd.date_range( start='2000-01-01', periods=11, freq='M' )[ [1,2,3,5,8,10] ]
        z = np.random.normal( size=(len(y), len(x)) )
        fig, ax = plt.subplots()
        pcolormesh(x,y,z, ax=ax)
        axis_month(ax)
        plt.show()
    """

    assert ( y is None and z is None ) or ( y is not None and z is not None ), "either provide one argument (x and y will be inferred), or three (provide both x and y)"
    if y is None and z is None:
        z = x
        if isinstance( z, pd.DataFrame ):
            x = z.columns
            y = z.index
        else:
            x = np.arange( z.shape[1] )
            y = np.arange( z.shape[0] )

    ax_was_None = ax is None
    assert len(x) == z.shape[1], f"Incompatible dimensions: z is {z.shape}, x is {x.shape} -- it should be {z.shape[1]}"
    assert len(y) == z.shape[0], f"Incompatible dimensions: z is {z.shape}, y is {y.shape} -- it should be {z.shape[0]}"
    def f(y):
        if isinstance( y[0], str ):
            yy = np.arange(len(y)+1) - .5
        elif isinstance( y[0], pd.Timestamp ) and len(y) > 1:
            u = matplotlib.dates.date2num(y)
            m = [ (a+b)/2 for a,b in zip(u[:-1],u[1:]) ]
            yy = [ u[0] - ( u[1] - u[0] ) / 2 ] + m + [ u[-1] + ( u[-1] - u[-2] ) / 2 ]
            yy = matplotlib.dates.num2date(yy)
        elif isinstance( y[0], pd.Timestamp ):
            yy = [ y[0] - datetime.timedelta(hours=12), y[0] + datetime.timedelta(hours=12) ]
        elif len(y) == 1:
            yy = [ y[0]-.5, y[0]+.5 ]
        else:
            u = y
            m = [ (a+b)/2 for a,b in zip(u[:-1],u[1:]) ]
            yy = [ u[0] - ( u[1] - u[0] ) / 2 ] + m + [ u[-1] + ( u[-1] - u[-2] ) / 2 ]
        return yy
    xx = f(x)
    yy = f(y)
    if ax_was_None:
        fig, ax = plt.subplots()
    p = ax.pcolormesh( xx, yy, z, **kwargs )
    if isinstance( y[0], str ):
        ax.set_yticks(np.arange(len(y)))
        ax.set_yticklabels(y)
    if isinstance( x[0], str ):
        ax.set_xticks(np.arange(len(x)))
        ax.set_xticklabels(x)
    if ax_was_None:
        plt.show()
    return p

def uniformize1(x: np.ndarray) -> np.ndarray:
    """Uniformize a 1-dimensional NumPy array"""
    assert len( x.shape ) == 1
    a = scipy.stats.rankdata( x, nan_policy = 'omit' )
    missing = np.isnan(x)
    a[:] = np.where( missing, np.NaN, a )
    y = (a - .5) / np.nanmax(a, axis=0)  # Between 0 and 1, excluding 0 and 1
    return y

def uniformize2(x: np.ndarray) -> np.ndarray:
    """Uniformize the columns of a 2-dimensional NumPy array"""
    assert len(x.shape) == 2
    y = x.copy()
    for i in range(x.shape[1]):
        y[:,i] = uniformize1( x[:,i] )
    return y

def uniformize(x):
    """
    Uniformize 1-dimensional (or the columns of a 2-dimensional) NumPy array or Pandas Series or DataFrame
    """
    if len(x.shape) == 1:
        if isinstance(x, pd.Series):
            y = uniformize1(x.values)
            y = pd.Series(y, index = x.index)
        else:
            y = uniformize1(x)
    else:
        assert len(x.shape) == 2, f"Expecting a 1- or 2-dimensional array, got a {len(x.shape)}-dimensional one..."
        if isinstance(x, pd.DataFrame):
            y = uniformize2(x.values)
            y = pd.DataFrame(y, index = x.index, columns = x.columns)
        else:
            y = uniformize2(x)
    return y

logging.basicConfig(
    format  = '%(asctime)-15s %(message)s',
    datefmt = '%Y-%m-%d %H:%M:%S',
    level   = logging.INFO,
)

def LOG(*args) -> None:
    """
    Print a message to stderr.
    You can also use logging.info() directly.
    """
    logging.info(*args)

In [ ]:
DPI = 100
plt.rcParams['figure.dpi'] = DPI

# Data

In [ ]:
# Toy datasets
X, y = sklearn.datasets.load_iris(return_X_y=True, as_frame=True)
X, y = sklearn.datasets.load_diabetes(return_X_y=True, as_frame=True)  # The data has been pre-processed: the qualitative variables have been replaced by avg(y)...
X, y = sklearn.datasets.load_digits(return_X_y=True, as_frame=True)
X, y = sklearn.datasets.load_linnerud(return_X_y=True, as_frame=True)
X, y = sklearn.datasets.load_wine(return_X_y=True, as_frame=True)            # For dimension reduction?
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)   # For dimension reduction?

In [ ]:
# Real-world datasets
if False: 
    d = sklearn.datasets.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))   # Text
    X, y = sklearn.datasets.fetch_california_housing( return_X_y = True, as_frame = True )             # Numeric data only
    d = sklearn.datasets.fetch_species_distributions()  # atypical dataset

In [ ]:
# Another housing dataset
if False: 

    housing = sklearn.datasets.fetch_openml(name="house_prices", as_frame=True)
    
    fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained' )
    ax.scatter(
        housing['data']['GrLivArea'],
        housing['target'],
        alpha = .1,
    )
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel( "Area (sq.ft.)" )
    ax.set_ylabel( "Price (USD)" )
    ax.set_title( "House prices" )
    plt.show()    

In [ ]:
# Ames housing dataset
!wget -nc https://github.com/INRIA/scikit-learn-mooc/raw/main/datasets/ames_housing_no_missing.csv
#ames = pd.read_csv( "https://github.com/INRIA/scikit-learn-mooc/raw/main/datasets/ames_housing_no_missing.csv" )
ames = pd.read_csv( "ames_housing_no_missing.csv" )

In [ ]:
# Financial data
ids = {
    'Apple':     'AAPL',
    'Microsoft': 'MSFT',
    'Amazon':    'AMZN',
    'Google':    'GOOG',
    'Meta':      'META',
    'Tesla':     'TSLA',
    'Berkshire': 'BRK-B',
    'Visa':      'V',
    'NVidia':    'NVDA',
}
stocks = {}
for label, id in ids.items():
    print( f"{label}: {id}" )
    stocks[label] = yf.Ticker(id).history(period="30y")

In [ ]:
d = data("diamonds")

In [ ]:
x = data("faithful").iloc[:,1]

In [ ]:
# World development indicators
indicators = wb.download(indicator='NY.GDP.MKTP.CD', country=['all'], start=1950, end=2023)
indicators = indicators.reset_index().pivot( index = 'country', columns = 'year', values = 'NY.GDP.MKTP.CD' )

Copilot suggests:

1. **World Development Indicators**: A comprehensive dataset from the World Bank containing global development data including population, income, social, economic, financial, natural resources, and environmental indicators.
3
2. **Gapminder World Data**: This dataset includes statistics about health, income, and the environment from several countries over the years and is great for creating visualizations that show changes over time.4

3. **Iris Flower Dataset**: A classic dataset for classification and visualization, containing measurements of iris flowers and their species. It's excellent for teaching scatter plots and classificatio5.

4. **United States Census Data**: Offers demographic information that can be used to visualize population distributions, income levels, and other social-economic factors across different (available via an API; requires registrati6ns.

5. **Global Earthquake Activity**: Data from the United States Geological Survey (USGS) can be used to create geospatial visualizations such as heat maps to show earthquake activity arounw the 7orld.

6. **Air Quality Index**: Data from environmental monitoring stations can be used to visualize pollution levels and compare air quality across different ciries or 8egions.

7. **Sports Statistics**: Datasets from sports leagues like NBA, NFL, or FIFA can be used to visualize player statistics, team performance, and other aports-rel9ted data.

8. **Stock Market Data**: Financial data such as stock prices over time can be visualized to show trends, volatility, andt her marke10 behaviors.

9. **COVID-19 Data**: Real-time data on COVID-19 cases, recoveries, and deaths can be used to create visualizations that show the spread tnd impact of11he pandemic.

10. **Climate Data**: Historical climate data such as temperature, rainfall, and sea level rise can be visualized to show patterns and changes in the environment.


# ChatGPT

In [ ]:
df = load_penguins()
df['island'].unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Assuming df is your pandas DataFrame
# Replace 'df' with the name of your actual DataFrame variable

# Create a figure and a set of subplots
fig, ax = plt.subplots()

# Unique markers for each island
markers = {'Biscoe': 'o', 'Dream': 's', 'Torgersen': '^'}

# Unique colors for each species
colors = {'Adelie': 'r', 'Chinstrap': 'g', 'Gentoo': 'b'}

# Plot each species
for species in df['species'].unique():
    for island in df['island'].unique():
        species_island_df = df[(df['species'] == species) & (df['island'] == island)]
        sns.regplot(x='bill_length_mm', y='bill_depth_mm', data=species_island_df, 
                    scatter_kws={'color': colors[species], 'marker': markers[island]},
                    line_kws={'color': colors[species], 'label': f'Linear fit for {species}'},
                    ax=ax)

# Set the labels for the axes
ax.set_xlabel('Bill Length (mm)')
ax.set_ylabel('Bill Depth (mm)')

# Add a legend
ax.legend()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Assuming df is your pandas DataFrame
# Replace 'df' with the name of your actual DataFrame variable

# Create a figure and a set of subplots
fig, ax = plt.subplots()

# Unique markers for each island
markers = {'Biscoe': 'o', 'Dream': 's', 'Torgersen': '^'}

# Unique colors for each species
colors = {'Adelie': 'r', 'Chinstrap': 'g', 'Gentoo': 'b'}

# Plot each species
for species in df['species'].unique():
    for island in df['island'].unique():
        species_island_df = df[(df['species'] == species) & (df['island'] == island)]
        sns.regplot(
            x = 'bill_length_mm', 
            y = 'bill_depth_mm', 
            data = species_island_df, 
            marker = markers[island],
            scatter_kws = {'color': colors[species]},
            line_kws    = {'color': colors[species], 'label': f'Linear fit for {species}'},
            ax = ax,
        )

# Set the labels for the axes
ax.set_xlabel('Bill Length (mm)')
ax.set_ylabel('Bill Depth (mm)')

# Add a legend
ax.legend()

# Show the plot
plt.show()

# Univariate, quantitative

## Boxplot

In [ ]:
np.random.seed(0)
x = np.random.normal( size=3000 )
x = np.exp(x) ** (1/2)
fig, ax = plt.subplots( figsize = (8,2), layout = 'constrained', dpi=DPI )
b = ax.boxplot(x, vert=False, patch_artist=True)
for u in b['boxes']:
    u.set_facecolor('lightblue')
    u.set_edgecolor('black')
for u in b['medians']:
    u.set_color('black')    
ax.set_yticks([])
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
plt.show()

## Histogram, density

In [ ]:
np.random.seed(0)
x = np.random.normal( size=3000 )
x = np.exp(x) ** (1/2)
for which in [0,1,2,3,4]:
    density = which >= 1
    fig, ax = plt.subplots(figsize = (4,4), layout = 'constrained', dpi=DPI)
    ax.hist(x, bins=20, facecolor='lightblue', edgecolor='tab:blue', density=density)
    ax.set_xlabel( "Value" )
    ax.set_ylabel( "Density" if density else "Count" )
    if which >= 2: 
        for side in ['left', 'top', 'right']:
            ax.spines[side].set_visible(False)
        ax.set_yticks([])
        ax.set_ylabel(None)
    if which >= 3:
        xs = np.linspace(x.min(), x.max(), 100)
        de = scipy.stats.gaussian_kde(x)
        ys = de(xs)
        ax.plot( xs, ys, linewidth = 5 )
    if which == 4: 
        ax.set_yscale('log')
        ax.set_ylim( 5e-4, 1 )
        ax.set_yticks([], minor=True)
        ax.set_yticks([])
    plt.show()

## Logarithmic scale

In [ ]:
n = 100_000
np.random.seed(0)
x1 = np.random.normal( size = n )
x2 = np.random.standard_t( size = n, df = 2 )
a = 4
x2 = x2[ np.abs(x2) < a ]
bins = np.linspace( -a, a, 20 )

xs = np.linspace(-a, a, 100)
y1 = scipy.stats.gaussian_kde(x1)(xs)
y2 = scipy.stats.gaussian_kde(x2)(xs)

for log in [False, True]:    
    fig, axs = plt.subplots( 1, 2, figsize = (8,3), dpi=DPI)
    axs[0].hist(x1, bins=bins, facecolor='lightblue', edgecolor='tab:blue', density=True)
    axs[1].hist(x2, bins=bins, facecolor='lightblue', edgecolor='tab:blue', density=True)
    for ax in axs: 
        for side in ['left', 'top', 'right']:
            ax.spines[side].set_visible(False)
            ax.set_yticks([])
            ax.set_ylabel(None)    
    if log: 
        for ax in axs:
            ax.set_yscale('log')
            ax.set_ylim( 3e-3, .6 )
        axs[0].plot( xs, y1, linewidth = 3 )
        axs[1].plot( xs, y2, linewidth = 3 )   
    plt.show()

In [ ]:
fig, ax = plt.subplots( figsize = (4,4), dpi = DPI )
ax.hist(x2, bins=bins, facecolor='lightblue', edgecolor='tab:blue', density=True)
for side in ['left', 'top', 'right']:
    ax.spines[side].set_visible(False)
    ax.set_yticks([])
    ax.set_ylabel(None)    
ax.set_yscale('log')
ax.set_ylim( 3e-3, .6 )
ax.plot( xs, y2, linewidth = 3 )   
plt.show()

## Quantile-quantile plot

In [ ]:
def qqplot(x, D = scipy.stats.norm(), ax=None):
    ax_was_None = ax is None
    n = len(x)
    ps = np.linspace( 1/(2*n), 1-1/(2*n), n )  # n points uniformly distributed in [0,1]
    xs = sorted(x)
    if ax_was_None: 
        fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained')

    ax.scatter( D.ppf(ps), xs, color = 'tab:cyan' )

    # Line through the quartiles
    i1 = n // 4
    i2 = (3*n) // 4
    ax.axline(
        ( D.ppf(ps[i1]), xs[i1] ),
        ( D.ppf(ps[i2]), xs[i2] ),
        color = 'black',
        zorder = +1,
    )
    ax.scatter( D.ppf(ps[i1]), xs[i1], color = 'black' )
    ax.scatter( D.ppf(ps[i2]), xs[i2], color = 'black' )

    ax.set_xlabel( "Theoretical quantiles" )
    ax.set_ylabel( "Sample quantiles" )
    if ax_was_None: 
        plt.show()

In [ ]:
n = 1_000
np.random.seed(0)
x = np.random.standard_t( size = n, df = 4 )

n = len(x)
D = scipy.stats.norm()
ps = np.linspace(0, 1, 2*n+1)[1::2]
ys = sorted(x)
xs = D.ppf(ps)
#plt.scatter( xs, ys )

fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = DPI )
qqplot(x, ax=ax)
ax.set_xlabel( "Gaussian quantiles" )
ax.set_ylabel( "Data quantiles" )
plt.show()

## Real data

In [ ]:
x = data("faithful").iloc[:,1]

## Numbers

In [ ]:
x.describe()

In [ ]:
{ 
    'dtype':                str(x.dtype),
    'observations':         len(x),
    'missing':              x.isnull().sum(),
    'mode':                 list(x.mode()),
    'mode occurrences':     ( x == x.mode()[0] ).sum(),
    'zeroes':               (x == 0).sum(),
    'mean':                 x.mean(), 
    'median':               x.median(), 
    'standard deviation':   x.std(),
    'inter-quartile range': x.quantile([.25,.75]).diff().iloc[-1],
    'skewness':             x.skew(),
    'kurtosis':             x.kurt(),
    'quantiles':            list( x.quantile( [.01, .05, .10, .25, .50, .75, .90, .95, .99] ) ),
    'min':                  x.min(),
    'max':                  x.max(),
}

## Histogram, density

In [ ]:
x.hist()
plt.show()
x.hist(bins=20)

In [ ]:
# Density
x.plot.hist( density = True, bins = 20 )
x.plot.density( linewidth = 5 )

In [ ]:
xs = np.linspace(x.min(), x.max(), 100)
de = scipy.stats.gaussian_kde(x)
ys = de(xs)
plt.plot( xs, ys )

In [ ]:
# Prettier plots
for add_density in [False, True]:
    for bins in [10, 20, 50]:
        fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained', dpi=DPI )
        ax.hist( x, bins = bins, density = True, color = 'lightblue' )
        if add_density:
            ax.plot( xs, ys, color = 'tab:blue', linewidth = 3 )
        ax.set_yticks([])
        ax.set_xlabel("Inter-eruption time")
        for side in ['left', 'right', 'top']: 
            ax.spines[side].set_visible(False)
        plt.show()

## Real data, with fat tails

In [ ]:
import yfinance as yf
x = yf.Ticker("^GSPC").history(period="30y")['Close']
x = np.log(x).diff().dropna()

In [ ]:
np.round( 100 * x.values, 2 )

In [ ]:
xs = np.linspace(x.min(), x.max(), 100)
de = scipy.stats.gaussian_kde(x)
ys = de(xs)

for log_scale in [False, True]:
    fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained', dpi=DPI )
    ax.hist( 100 * x, bins = 100, density = True, color = 'lightblue' )
    ax.plot( 100 * xs, ys/100, color = 'tab:blue', linewidth = 3 )
    ax.set_xlabel("Daily logarithmic returns (%)")
    for side in ['left', 'right', 'top']: 
        ax.spines[side].set_visible(False)
    if log_scale:
        ax.set_yscale('log')
        ax.set_ylim(1e-4,1)
    else:
        ax.set_yticks([])
    plt.show()

In [ ]:
def qqplot(x, D = scipy.stats.norm(), ax=None):
    ax_was_None = ax is None
    n = len(x)
    ps = np.linspace( 1/(2*n), 1-1/(2*n), n )  # n points uniformly distributed in [0,1]
    xs = sorted(x)
    if ax_was_None: 
        fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained')

    ax.scatter( D.ppf(ps), xs )

    # Line through the quartiles
    i1 = n // 4
    i2 = (3*n) // 4
    ax.axline(
        ( D.ppf(ps[i1]), xs[i1] ),
        ( D.ppf(ps[i2]), xs[i2] ),
        color = 'black',
        zorder = -1,
    )
    
    ax.set_xlabel( "Theoretical quantiles" )
    ax.set_ylabel( "Sample quantiles" )
    if ax_was_None: 
        plt.show()

fig, axs = plt.subplots( 1, 2, figsize = (7,3), layout = 'constrained', dpi=DPI )
qqplot(x, ax = axs[0] )
qqplot(x, scipy.stats.t(df=3), ax = axs[1] )
axs[0].set_title( "Comparison with\n a Gaussian distribution" )
axs[1].set_title( "Comparison with\n a Student T(df=4) distribution" )
plt.show()

## What to look for in histograms

In [ ]:
# Examples of distributions; with a linear or log-scale

for log in [False, True]: 
    fig, axs = plt.subplots(2,3, figsize=(8,4.5), layout='constrained', dpi=DPI)
    xs = np.linspace(-3,3,100)
    axs[0,0].plot( xs, scipy.stats.norm.pdf(xs) )
    axs[0,1].plot( xs, scipy.stats.t.pdf(xs, df=4) )
    axs[0,2].plot( xs, scipy.stats.cauchy.pdf(xs) )
    axs[1,0].plot( xs, scipy.stats.laplace.pdf(xs) )
    axs[1,1].plot( xs, scipy.stats.skewnorm.pdf(xs, a=3, loc=-1) )
    xs = np.linspace(-1e-6,3,100)
    axs[1,2].plot( xs, scipy.stats.expon.pdf(xs) )
    axs[0,0].set_title("Gaussian")
    axs[0,1].set_title("Student")
    axs[0,2].set_title("Cauchy")
    axs[1,0].set_title("Laplace")
    axs[1,2].set_title("Exponential")
    for ax in axs.flatten():
        for side in ['left','right','top']:
            ax.spines[side].set_visible(False)
        ax.set_yticks([])
        if log: 
            ax.set_yscale('log')
    plt.show()

In [ ]:
# Data characteristics to pay attention to: zero-inflation, outliers, fat tails, skewness, multimodality    
for log in [False, True]: 
        
    fig, axs = plt.subplots(2,3, figsize=(8,4.5), layout='constrained', dpi=DPI)
    x = scipy.stats.norm.rvs(size=10_000)
    x[:1000] = 0
    axs[0,0].hist(x, bins=30)
    
    x = scipy.stats.norm.rvs(size=10_000)
    x[:150] = np.random.choice( [5, -6, -4, 4, 8], size=150 )
    axs[0,1].hist(x, bins=50)
    
    x = scipy.stats.laplace.rvs(size=10_000)
    axs[0,2].hist(x, bins=50, density = True)
    #xs = np.linspace(-3,3,100)
    #axs[0,2].plot( xs, scipy.stats.laplace.pdf(xs) )
    
    x = scipy.stats.skewnorm.rvs(a=3, loc=-1, size=10_000)
    axs[1,0].hist(-x, bins=50, density = True)
    
    x = scipy.stats.skewnorm.rvs(a=3, loc=-1, size=10_000)
    axs[1,1].hist(x, bins=50, density = True)
    
    x1 = np.random.normal(size=6000) - 2
    x2 = np.random.normal(size=4000) + 2
    x = np.concatenate([x1,x2])
    axs[1,2].hist(x, bins=50, density = True)
    
    axs[0,0].set_title( "Zero inflation" )
    axs[0,1].set_title( "Outliers" )
    axs[0,2].set_title( "Fat tails" )
    axs[1,0].set_title( "Skewness < 0" )
    axs[1,1].set_title( "Skewness > 0" )
    axs[1,2].set_title( "Multimodality" )
    
    for ax in axs.flatten():
        for side in ['left','right','top']:
            ax.spines[side].set_visible(False)
        ax.set_yticks([])
        if log: 
            ax.set_yscale('log')
    
    plt.show()

## Numbers (plots explaining what they are)

In [ ]:
# Mean, median, mode, standard deviation, quantiles, IQR

fig, axs = plt.subplots(2,3, figsize=(8,4.5), layout='constrained')
xs = np.linspace(-3,3,100)
axs[0,0].fill_between( xs, 0*xs, scipy.stats.norm.pdf(xs), color='lightblue')
axs[0,0].axvline( 0, linewidth = 2, color = 'black', linestyle = ':' )

D = scipy.stats.skewnorm(a=40, loc=-1)
D = scipy.stats.gamma(a=2, scale=2)
#D = scipy.stats.expon(scale=5)
xs = np.linspace(0,10,100)
i = xs <= D.median()
axs[0,1].fill_between( xs[i],  D.pdf(xs[i]),  color = 'peachpuff')
axs[0,1].fill_between( xs[~i], D.pdf(xs[~i]), color = 'lightblue')
#axs[0,1].axvline( D.mean(),   linewidth = 2, color = 'black', linestyle = ':' )
axs[0,1].axvline( D.median(), linewidth = 2, color = 'black' )

i = np.argmax(D.pdf(xs))
axs[0,2].fill_between( xs, D.pdf(xs), color = 'lightblue')
axs[0,2].scatter( [ xs[i] ], [ D.pdf(xs[i]) ], s = 100, color = 'black' )

xs = np.linspace(-3,3,100)
D = scipy.stats.norm()
axs[1,0].fill_between( xs, 0*xs, D.pdf(xs), color='lightblue')
hl = .5
axs[1,0].arrow( -1+hl, D.pdf(1), dx=2-hl,  dy=0, length_includes_head = True, width = .01, head_length = hl, color = 'black' )
axs[1,0].arrow(  1-hl, D.pdf(1), dx=-2+hl, dy=0, length_includes_head = True, width = .01, head_length = hl, color = 'black' )

D = scipy.stats.gamma(a=2, scale=2)
xs = np.linspace(0,10,100)
qs = np.linspace(0,1,11)
qs = D.ppf(qs)
for j, (left, right)in enumerate(zip( qs[:-1], qs[1:] )):
    i = ( left <= xs ) & ( xs <= right )
    axs[1,1].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Pastel2(j % 8))

qs = np.linspace(0,1,5)
qs = D.ppf(qs)
for j, (left, right)in enumerate(zip( qs[:-1], qs[1:] )):
    i = ( left <= xs ) & ( xs <= right )
    #axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Paired([5,4,0,1])[j])
    #axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Set3([3,11,7,2])[j])
    #axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Pastel1([0,4,7,1])[j])
    axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.turbo_r([.2, .4, .6, .8], alpha=.4)[j] )
x1 = D.ppf(.25)
x2 = D.ppf(.75)
y = min( D.pdf(x1), D.pdf(x2) )
hl = .8
axs[1,2].arrow( x1+hl, y, dx=x2-x1-hl, dy=0, length_includes_head = True, width = .005, head_length = hl, color = 'black' )
axs[1,2].arrow( x2-hl, y, dx=x1-x2+hl, dy=0, length_includes_head = True, width = .005, head_length = hl, color = 'black' )

axs[0,0].set_title("Mean")
axs[0,1].set_title("Median")
axs[0,2].set_title("Mode")
axs[1,0].set_title("Standard Deviation")
axs[1,1].set_title("Quantiles")
axs[1,2].set_title("Inter-quartile range (IQR)")
for ax in axs.flatten():
    for side in ['left','right','top']:
        ax.spines[side].set_visible(False)
    ax.set_yticks([])
plt.show()

In [ ]:
# Use a Beta distribution instead...

# Mean, median, mode, standard deviation, quantiles, IQR

D = scipy.stats.beta(a=1.5,b=3)
N = scipy.stats.norm()

fig, axs = plt.subplots(2,3, figsize=(8,4.5), layout='constrained', dpi=DPI)
xs = np.linspace(-3,3,100)
axs[0,0].fill_between( xs, 0*xs, N.pdf(xs), color='lightblue')
axs[0,0].axvline( N.mean(), linewidth = 2, color = 'black', linestyle = ':' )

xs = np.linspace(0,1,100)

i = xs <= D.median()
axs[0,1].fill_between( xs[i],  D.pdf(xs[i]),  color = 'peachpuff')
axs[0,1].fill_between( xs[~i], D.pdf(xs[~i]), color = 'lightblue')
#axs[0,1].axvline( D.mean(),   linewidth = 2, color = 'black', linestyle = ':' )
axs[0,1].axvline( D.median(), linewidth = 2, color = 'black' )

i = np.argmax(D.pdf(xs))
axs[0,2].fill_between( xs, D.pdf(xs), color = 'lightblue')
axs[0,2].scatter( [ xs[i] ], [ D.pdf(xs[i]) ], s = 100, color = 'black' )

xs = np.linspace(-3,3,100)

axs[1,0].fill_between( xs, 0*xs, N.pdf(xs), color='lightblue')
hl = .5
axs[1,0].arrow( -1+hl, N.pdf(1), dx=2-hl,  dy=0, length_includes_head = True, width = .01, head_length = hl, color = 'black' )
axs[1,0].arrow(  1-hl, N.pdf(1), dx=-2+hl, dy=0, length_includes_head = True, width = .01, head_length = hl, color = 'black' )

xs = np.linspace(0,1,100)

qs = np.linspace(0,1,11)
qs = D.ppf(qs)
for j, (left, right)in enumerate(zip( qs[:-1], qs[1:] )):
    i = ( left <= xs ) & ( xs <= right )
    axs[1,1].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Pastel2(j % 8))

qs = np.linspace(0,1,5)
qs = D.ppf(qs)
for j, (left, right)in enumerate(zip( qs[:-1], qs[1:] )):
    i = ( left <= xs ) & ( xs <= right )
    #axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Paired([5,4,0,1])[j])
    #axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Set3([3,11,7,2])[j])
    #axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.Pastel1([0,4,7,1])[j])
    axs[1,2].fill_between( xs[i],  D.pdf(xs[i]),  color = plt.cm.turbo_r([.2, .4, .6, .8], alpha=.4)[j] )
x1 = D.ppf(.25)
x2 = D.ppf(.75)
y = min( D.pdf(x1), D.pdf(x2) )
hl = .08
axs[1,2].arrow( x1+hl, y, dx=x2-x1-hl, dy=0, length_includes_head = True, width = .05, head_length = hl, color = 'black' )
axs[1,2].arrow( x2-hl, y, dx=x1-x2+hl, dy=0, length_includes_head = True, width = .05, head_length = hl, color = 'black' )

axs[0,0].set_title("Mean")
axs[0,1].set_title("Median")
axs[0,2].set_title("Mode")
axs[1,0].set_title("Standard Deviation")
axs[1,1].set_title("Quantiles")
axs[1,2].set_title("Inter-quartile range (IQR)")
for ax in axs.flatten():
    for side in ['left','right','top']:
        ax.spines[side].set_visible(False)
    ax.set_yticks([])
plt.show()

In [ ]:
pd.Series( D.rvs(1000) ).skew()


In [ ]:
# Skewness

a,b = 3, 1.5
D1 = scipy.stats.beta(a=a,b=b)
D2 = scipy.stats.beta(a=b,b=a)

N = scipy.stats.norm()

fig, axs = plt.subplots(1,3, figsize=(8,2.25), layout='constrained', dpi=DPI)
xs = np.linspace(-3,3,100)
axs[1].fill_between( xs, 0*xs, N.pdf(xs), color='lightblue')

xs = np.linspace(0,1,100)
axs[0].fill_between( xs, D1.pdf(xs),  color = 'lightblue')
axs[2].fill_between( xs, D2.pdf(xs),  color = 'lightblue')

axs[0].set_title( f"Skewness = {pd.Series( D1.rvs(1_000_000) ).skew():.1f}" )
axs[2].set_title( f"Skewness = {pd.Series( D2.rvs(1_000_000) ).skew():.1f}" )
axs[1].set_title( f"Skewness = 0" )

for ax in axs.flatten():
    ax.axis('off')
    for side in ['left','right','top']:
        ax.spines[side].set_visible(False)
    ax.set_yticks([])
plt.show()

In [ ]:
# Skewness

D0 = scipy.stats.beta(a=2,b=2)
D0 = scipy.stats.beta(a=1.5,b=1.5)
D1 = scipy.stats.norm()
D2 = scipy.stats.cauchy()

fig, axs = plt.subplots(1,3, figsize=(8,2.25), layout='constrained', dpi=DPI)

xs = np.linspace(0,1,100)
axs[0].fill_between( xs, D0.pdf(xs),  color = 'lightblue')

xs = np.linspace(-3,3,100)
axs[1].fill_between( xs, 0*xs, D1.pdf(xs), color='lightblue')

xs = np.linspace(-10,10,100)
axs[2].fill_between( xs, D2.pdf(xs),  color = 'lightblue')

axs[0].set_title( f"(Excess) kurtosis = {pd.Series( D0.rvs(1_000_000) ).kurt():.1f}" )
axs[1].set_title( f"(Excess) kurtosis = {pd.Series( D1.rvs(1_000_000) ).kurt():.1f}" )
axs[2].set_title( f"(Excess) kurtosis = {pd.Series( D2.rvs(1_000_000) ).kurt():.1f}" )

axs[0].set_title( f"(Excess) kurtosis = -1" )
axs[1].set_title( f"(Excess) kurtosis = 0" )
axs[2].set_title( f"(Excess) kurtosis > 0" )

for ax in axs.flatten():
    ax.axis('off')
    for side in ['left','right','top']:
        ax.spines[side].set_visible(False)
    ax.set_yticks([])
plt.show()

In [ ]:
# TODO: Compare the tails

In [ ]:
# TODO: extra material: fitting a distribution; fitting the tails

  data types
  Numbers: mean, median, sd, IQR, skew, kurt, quantiles, missing, mode, min, max, number of unique values (constant/id), duplicate rows
  zero inflation, magic numbers
  boxplot, histogram, density, log-scale, fitted distribution
  comparison of the tails, fit the tails, quantile-quantile plot
  outliers

# Qualitative variables

## Univariate

In [ ]:
x = data("diamonds")['clarity']

In [ ]:
x.values

In [ ]:
x.describe()

In [ ]:
{
    'observations':  len(x),
    'missing':       x.isnull().sum(),
    'Unique values': len(np.unique(x.values)),
    'Mode':          Counter(x).most_common(1)[0][0],
    'Mode count':    Counter(x).most_common(1)[0][1],
}

In [ ]:
c = Counter(x)
c = pd.DataFrame( { 
    'clarity': c.keys(),
    'count':   c.values(),
} ).sort_values('count' )
#fig, ax = plt.subplots( figsize = (6,2.5), layout = 'constrained', dpi = DPI )
fig, ax = plt.subplots( figsize = (3,3), layout = 'constrained', dpi = 100 )
ax.barh( c['clarity'], c['count'] )
ax.set_xlabel( "Count" )
ax.set_title("Clarity")
for side in ['left', 'top', 'right']:
    ax.spines[side].set_visible(False)
plt.show()

In [ ]:
# piechart, doughnut, treemap

def doughnut_plot(x, ax=None, pie = False, linewidth = 7):
    assert len(x.shape) == 1, "x should be 1-dimensional"
    assert isinstance( x, pd.Series )
    ax_was_None = ax is None
    if ax_was_None:
        fig, ax = plt.subplots()
    ax.pie(
        x,
        labels = x.index,
        #explode=.05*np.ones(len(x)),   # Bad idea: looks irregular if there are very large and very small groups
        wedgeprops = { 'linewidth' : linewidth, 'edgecolor' : 'white' },   # Same effect as "explode", without the problems
        # autopct='%1.1f%%', pctdistance=.85,   # Show the percentages
    )
    circle = plt.Circle(xy=(0, 0), radius=0.75, facecolor='white')
    if not pie: 
        ax.add_artist(circle)
    if ax_was_None:
        plt.show()

fig, axs = plt.subplots( 1, 3, figsize = (12,4), layout = 'constrained', dpi=100)
doughnut_plot( c.set_index("clarity")['count'], linewidth = 2, ax = axs[0], pie = True )
doughnut_plot( c.set_index("clarity")['count'], linewidth = 2, ax = axs[1] )

col = cycle( plt.get_cmap("tab10").colors )
col = islice(col,c.shape[0])
col = list(col)
squarify.plot(
    sizes = c['count'], 
    label = c['clarity'], 
    color = col, 
    alpha = .8, 
    text_kwargs={'color': 'white'}, 
    pad = True,
    ax = axs[2],
)
axs[2].set_aspect(1)
axs[2].axis('off')

plt.show()

In [ ]:
# Another example, with words
d = sklearn.datasets.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
x = pd.Series( np.concatenate( [
    re.sub( r'[^a-z]', ' ', u.lower() ).split()
    for u in d['data'] 
] ) )

In [ ]:
{
    'observations':  len(x),
    'missing':       x.isnull().sum(),
    'Unique values': len(np.unique(x.values)),
    'Mode':          Counter(x).most_common(1)[0][0],
    'Mode count':    Counter(x).most_common(1)[0][1],
}

In [ ]:
c = Counter(x)
c = pd.DataFrame( { 
    'word': c.keys(),
    'count':   c.values(),
} ).sort_values('count', ascending = False )
fig, ax = plt.subplots( figsize = (3,3), layout = 'constrained', dpi = 100 )
ax.scatter( 1+np.arange(c.shape[0]), c['count'] )
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Words")
ax.set_ylabel("Counts")
plt.show()

In [ ]:
# Truncating the data does not give anything usable: the largest category is still the "rest"...
k = 10
cc = pd.concat( [ 
    c.iloc[:k,:],
    pd.DataFrame( { 
        'word': [ '(rest)' ],
        'count': c.iloc[k:,:]['count'].sum(),
    } ),
] ).iloc[::-1,:]
fig, ax = plt.subplots( figsize = (3,3), layout = 'constrained', dpi = 100 )
ax.barh( cc['word'], cc['count'] )
ax.set_xlabel( "Count" )
ax.set_title("Word")
for side in ['left', 'top', 'right']:
    ax.spines[side].set_visible(False)
plt.show()

In [ ]:
if False:  # To not try to plot a pie chart with 100_000 categories...
    
    fig, axs = plt.subplots( 1, 3, figsize = (12,4), layout = 'constrained', dpi=100)
    doughnut_plot( c.set_index("word")['count'], linewidth = 0, ax = axs[0], pie = True )
    doughnut_plot( c.set_index("word")['count'], linewidth = 0, ax = axs[1] )
    
    col = cycle( plt.get_cmap("tab10").colors )
    col = islice(col,c.shape[0])
    col = list(col)
    squarify.plot(sizes=c['count'], label=c['word'], color=col, alpha = .8, text_kwargs={'color': 'white'}, ax = axs[2])
    axs[2].set_aspect(1)
    axs[2].axis('off')
    
    plt.show()

## Bivariate

In [ ]:
d = data("diamonds")[['color','clarity']]
d.head()

In [ ]:
c = d.copy()
c['count'] = 1
c = c.groupby(['color','clarity']).sum().reset_index().pivot( index = 'color', columns = 'clarity', values = 'count' )
c

In [ ]:
def stacked_bars(c, ax = None, proportion = False):
    ax_was_None = ax is None
    if ax_was_None: 
        fig, ax = plt.subplots( layout = 'constrained' )
    s = 0 * c.iloc[:,0]
    S = c.T.sum()
    if not proportion: 
        S = 1 + 0 * S
    for u in c.columns:
        ax.bar( c.index, c[u] / S, bottom = s, label = u )
        s += c[u] / S
    ax.legend( reverse = True, title = c.columns.name )
    ax.set_xlabel( c.index.name )
    ax.set_ylabel( "Proportion of observations" if proportion else "Number of observations" )
    if ax_was_None: 
        plt.show()

fig, ax = plt.subplots( figsize = (4, 4), layout = 'constrained', dpi = 100 )
stacked_bars(c, ax=ax)
ax.set_title( "Diamond clarity and colour" )
plt.show()

fig, axs = plt.subplots( 2, 2, figsize = (16,9) )
stacked_bars( c,   ax = axs[0,0] )
stacked_bars( c,   ax = axs[0,1], proportion = True )
stacked_bars( c.T, ax = axs[1,0] )
stacked_bars( c.T, ax = axs[1,1], proportion = True )
plt.show()

In [ ]:
# There is a mosaic plot, but the default colours are really, really bad...
fig, axs = plt.subplots( 1, 2, figsize = (8,4), layout = 'constrained', dpi = 100)
statsmodels.graphics.mosaicplot.mosaic( d, ['color', 'clarity'], statistic = False, ax = axs[0] )
statsmodels.graphics.mosaicplot.mosaic( d, ['color', 'clarity'], statistic = True,  ax = axs[1] )
plt.show()

In [ ]:
colours = matplotlib.cm.Pastel1.colors
colours = dict( zip( np.unique( d.iloc[:,1] ), colours ) )
properties = lambda u: { 'color': colours[u[1]] }
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100)
statsmodels.graphics.mosaicplot.mosaic( 
    d, 
    list(d.columns), 
    #statistic = True,
    properties = properties,
    ax = ax,
)
plt.show()

In [ ]:
def _statistical_coloring(data, index = None, cmap = matplotlib.cm.RdBu):
    """
    Copied from statsmodels.graphics.mosaicplot._statistical_coloring, except for the colours
    """
    data = statsmodels.graphics.mosaicplot._normalize_data(data, index)
    categories_levels = statsmodels.graphics.mosaicplot._categories_level(list(data.keys()))
    Nlevels = len(categories_levels)
    total = 1.0 * sum(v for v in data.values())
    # count the proportion of observation
    # for each level that has the given name
    # at each level
    levels_count = []
    for level_idx in range(Nlevels):
        proportion = {}
        for level in categories_levels[level_idx]:
            proportion[level] = 0.0
            for key, value in data.items():
                if level == key[level_idx]:
                    proportion[level] += value
            proportion[level] /= total
        levels_count.append(proportion)
    # for each key I obtain the expected value
    # and it's standard deviation from a binomial distribution
    # under the hipothesys of independence
    expected = {}
    for key, value in data.items():
        base = 1.0
        for i, k in enumerate(key):
            base *= levels_count[i][k]
        expected[key] = base * total, np.sqrt(total * base * (1.0 - base))
    # now we have the standard deviation of distance from the
    # expected value for each tile. We create the colors from this
    sigmas = dict((k, (data[k] - m) / s) for k, (m, s) in expected.items())
    
    props = {}
    for key, dev in sigmas.items():
        #red = 0.0 if dev < 0 else (dev / (1 + dev))
        #blue = 0.0 if dev > 0 else (dev / (-1 + dev))
        #green = (1.0 - red - blue) / 2.0
        colour = cmap( np.clip(dev,-2,2)/8 + .5 )
        hatch = 'x' if dev > 2 else 'o' if dev < -2 else ''
        props[key] = {'color': colour, 'hatch': hatch}
    return props

fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100)
statsmodels.graphics.mosaicplot.mosaic( 
    d, 
    list(d.columns), 
    #statistic = True,
    properties = _statistical_coloring(d, list(d.columns)),
    ax = ax,
)
plt.show()

## 2 numeric variables + 1 or 2 qualitative

In [ ]:
d = load_penguins()
d.groupby(['species','island']).count()['year'].reset_index().pivot( index = 'species', columns = 'island', values = 'year' )
#sns.pairplot(d)

islands = sorted( d['island'].unique() )
species = sorted( d['species'].unique() )
sex     = d['sex'].dropna().unique()

a = 'bill_length_mm'
b = 'bill_depth_mm'

d.head()

In [ ]:
colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
for line in [False, True]:
    fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
    for j, (k,v) in enumerate(d.groupby('species')):
        x, y = v[a], v[b]
        i = np.isfinite(x) & np.isfinite(y)
        x, y = x[i], y[i]
        ax.scatter( x, y, label = k, color = colours[j], alpha = .5 )
        if line: 
            model = sklearn.linear_model.LinearRegression()
            model.fit( pd.DataFrame(x.values), y )
            #ax.axline( ( x.mean(), y.mean() ), slope = model.coef_[0], color = colours[j] )
            xs = np.array( [ x.min(), x.max() ] )
            ys = model.coef_[0] * ( xs - x.mean() ) + y.mean()
            ax.plot( xs, ys, color = colours[j], linewidth = 5 )
    ax.set_xlabel(a)
    ax.set_ylabel(b)
    #ax.set_aspect(1)
    ax.legend()
    plt.show()


In [ ]:
def enlarge(a,b,epsilon=.02): 
    d = epsilon*(b-a)
    return a-d, b+d
    
fig, axs = plt.subplots( 1, 3, figsize=(9,3), layout = 'constrained', dpi = 100)
for i, (k,v) in enumerate(d.groupby('species')):
    ax = axs[i]
    ax.scatter( d[a], d[b], color = 'lightgrey', s = 1 )
    ax.scatter( v[a], v[b] )
    ax.set_title(k)
for ax in axs:
    ax.set_xlim( *enlarge( d[a].min(), d[a].max() ) )
    ax.set_ylim( *enlarge( d[b].min(), d[b].max() ) )
    ax.set_xlabel(a)
    ax.set_ylabel(b)
plt.show()

In [ ]:
colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['o', 's', '^']
colours = dict( zip( species, colours ) )
markers = dict( zip( islands, markers ) )

scale = .3
fig, ax = plt.subplots(figsize = (16*scale,9*scale), layout = 'constrained', dpi = 100)
for island in islands:
    for s in species: 
        i1 = d['island'] == island
        i2 = d['species'] == s        
        ax.scatter( d[i1&i2][a], d[i1&i2][b], color = colours[s], marker = markers[island], alpha = .5)

for s in species: 
    i1 = np.isfinite( d[a] ) & np.isfinite( d[b] )
    i2 = d['species'] == s        
    x, y = d[i1&i2][a], d[i1&i2][b]
    model = sklearn.linear_model.LinearRegression()
    model.fit( pd.DataFrame(x.values), y )
    xs = np.array( [ x.min(), x.max() ] )
    ys = model.coef_[0] * ( xs - x.mean() ) + y.mean()
    ax.plot( xs, ys, color = colours[s], linewidth = 5 )
        
ax.set_xlabel(a)
ax.set_ylabel(b)

legend_elements = [
    matplotlib.patches.Patch(facecolor=list(colours.values())[0], edgecolor='w', label=list(colours.keys())[0]),
    matplotlib.patches.Patch(facecolor=list(colours.values())[1], edgecolor='w', label=list(colours.keys())[1]),
    matplotlib.patches.Patch(facecolor=list(colours.values())[2], edgecolor='w', label=list(colours.keys())[2]),
    matplotlib.lines.Line2D([0], [0], marker=list(markers.values())[0], color='w', label=list(markers.keys())[0], markerfacecolor='black', markersize=10),
    matplotlib.lines.Line2D([0], [0], marker=list(markers.values())[1], color='w', label=list(markers.keys())[1], markerfacecolor='black', markersize=10),
    matplotlib.lines.Line2D([0], [0], marker=list(markers.values())[2], color='w', label=list(markers.keys())[2], markerfacecolor='black', markersize=10),
]
ax.legend(
    handles = legend_elements, 
    bbox_to_anchor = (1, 0.8),
)

plt.show()

In [ ]:
for which in [0,1]: 
        
    fig, axs = plt.subplots( 3, 3, figsize=(9,7), layout = 'constrained', dpi = 100 )
    for i, island in enumerate(islands):
        for j, s in enumerate(species): 
            ax = axs[i,j]
            i1 = d['island'] == island
            i2 = d['species'] == s
            if which == 1:
                for se in sex:
                    i3 = d['sex'] == se
                    ax.scatter( d[i1&i2&i3][a], d[i1&i2&i3][b], alpha = .5, label = se)
                if i == 0 and j == 0:
                    ax.legend()
            else: 
                ax.scatter( d[i1&i2][a], d[i1&i2][b], alpha = .5 )
            if i == 0:
                ax.set_title( s )
            if j == 2:
                pass
                ax.text( 
                    1.01, .5, island, 
                    transform = ax.transAxes, rotation = -90, ha='left', va='center',
                    fontsize =   plt.rcParams['axes.titlesize'],
                    fontweight = plt.rcParams['axes.titleweight'],
                )
    for ax in axs.flatten():
        ax.set_xlim( *enlarge( d[a].min(), d[a].max() ) )
        ax.set_ylim( *enlarge( d[b].min(), d[b].max() ) )
        ax.set_xlabel(a)
        ax.set_ylabel(b)
    plt.show()

## Example: Gapminder

In [ ]:
x = np.array([[1,2,3],[4,5,6]])
x.sum(axis=0)

In [ ]:
d = gapminder
display(d) 

d.groupby( ['country','year'] ).count()['pop'].reset_index().pivot(index = 'country', columns = 'year', values = 'pop' )
years = np.array( sorted( d['year'].unique() ) )

In [ ]:
fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
ax.scatter( gapminder['gdpPercap'], gapminder['lifeExp'], alpha = .2 )
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xscale('log')
plt.show()

fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
p = ax.scatter( gapminder['gdpPercap'], gapminder['lifeExp'], alpha = 1, c = gapminder['year'], cmap = 'turbo_r' )
ax.scatter( gapminder['gdpPercap'], gapminder['lifeExp'], c = 'white' )
ax.scatter( gapminder['gdpPercap'], gapminder['lifeExp'], alpha = .5, c = gapminder['year'], cmap = 'turbo_r' )
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xscale('log')
c = fig.colorbar(p)
plt.show()

In [ ]:
d = gapminder
fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
for k, v in d.groupby('continent'):
    ax.scatter( v['gdpPercap'], v['lifeExp'], alpha = .3, label = k )
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xscale('log')
leg = ax.legend()
for h in leg.legend_handles:
    h.set_alpha(1)
plt.show()

In [ ]:
d = gapminder.copy()
d['gdpPercap'] = np.log10( d['gdpPercap'] )
colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
for i, (k,v) in enumerate( d.groupby('continent') ):
    ax.scatter( v['gdpPercap'], v['lifeExp'], alpha = .1, s = 10, label = k )
    V = v[['gdpPercap','lifeExp']].cov()
    m = v[['gdpPercap','lifeExp']].mean()
    add_ellipse( V, m, radius = 3, color = colours[i], ax=ax )  # This does not work with log scales...
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xticks( [3,4,5], [r'$10^3$', r'$10^4$', r'$10^5$'] )
u = np.log10( np.outer( np.arange(1,10), 10**np.arange(2,6) ).T.flatten() )
u = u[ u > d['gdpPercap'].min() ]
u = u[ u < d['gdpPercap'].max() ]
ax.set_xticks( u, minor=True )
#ax.set_xscale('log')
leg = ax.legend( markerscale = 2 )  # Alternatively, reset the sizes in the loop below
for h in leg.legend_handles:
    h.set_alpha(1)
    h._sizes = [50]
plt.show()

In [ ]:
d = gapminder.copy()
d['gdpPercap'] = np.log10( d['gdpPercap'] )

#colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
n = len(d['year'].unique())
colours = matplotlib.cm.turbo_r( np.linspace(0,1,n) )

fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
for i, (k,v) in enumerate( d.groupby('year') ):
    ax.scatter( v['gdpPercap'], v['lifeExp'], alpha = .2, s = 10, label = k, color = colours[i], zorder = -3 )
    V = v[['gdpPercap','lifeExp']].cov()
    m = v[['gdpPercap','lifeExp']].mean()
    add_ellipse( V, m, radius = 3, color = colours[i], ax=ax )  # This does not work with log scales...
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xticks( [3,4,5], [r'$10^3$', r'$10^4$', r'$10^5$'] )
u = np.log10( np.outer( np.arange(1,10), 10**np.arange(2,6) ).T.flatten() )
u = u[ u > d['gdpPercap'].min() ]
u = u[ u < d['gdpPercap'].max() ]
ax.set_xticks( u, minor=True )
#ax.set_xscale('log')
leg = ax.legend( markerscale = 2, fontsize = 8, reverse = True )
for h in leg.legend_handles:
    h.set_alpha(1)
    #h._sizes = [50]
leg.legend_handles = leg.legend_handles[::-1]
plt.show()

In [ ]:
d = gapminder
ylim = enlarge( d['lifeExp'].min(), d['lifeExp'].max(), .04 )
xlim = d['gdpPercap'].min(), d['gdpPercap'].max()
xlim = np.log(xlim)
xlim = enlarge(*xlim)
xlim = np.exp(xlim)

i = gapminder['year'] == years.max()
d = gapminder[i]

def size(d): 
    return np.sqrt(d['pop'])/50

fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
ax.scatter( d['gdpPercap'], d['lifeExp'], s = 200, alpha = .7 )
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xscale('log')
ax.set_title( years.max() )
plt.show()

fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
ax.scatter( d['gdpPercap'], d['lifeExp'], s = size(d), alpha = .7 )
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xscale('log')
ax.set_title( years.max() )

legend_elements = [
    matplotlib.lines.Line2D([0], [0], marker='o', color='w', label="10m",  markerfacecolor='black', markersize = np.sqrt( np.sqrt(10_000_000)/50 ) ),
    matplotlib.lines.Line2D([0], [0], marker='o', color='w', label="100m", markerfacecolor='black', markersize = np.sqrt( np.sqrt(100_000_000)/50) ),
    matplotlib.lines.Line2D([0], [0], marker='o', color='w', label="1b",   markerfacecolor='black', markersize = np.sqrt( np.sqrt(1_000_000_000)/50) ),
]
ax.legend(handles=legend_elements, title = "Population")

plt.show()

fig, ax = plt.subplots(figsize = (4,3), layout = 'constrained', dpi = 100)
for k, v in d.groupby('continent'):
    ax.scatter( v['gdpPercap'], v['lifeExp'], s = size(v), alpha = .7, label = k )
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel( "GDP per capita (USD)" )
ax.set_ylabel( "Life expectancy (years)" )
ax.set_xscale('log')
ax.set_title( years.max() )
leg = ax.legend()
for h in leg.legend_handles:
    h.set_sizes([100.])
plt.show()

In [ ]:
scale = 1
fig, axs = plt.subplots(3,4, figsize = (scale*16,scale*9), layout = 'constrained', dpi=100)
for i, year in enumerate(years):
    ax = axs.flatten()[i]
    j = gapminder['year'] == year
    d = gapminder[j]
    for k, v in d.groupby('continent'):
        ax.scatter( v['gdpPercap'], v['lifeExp'], s = size(v), alpha = .7, label = k )
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_xlabel( "GDP per capita (USD)" )
    ax.set_ylabel( "Life expectancy (years)" )
    ax.set_xscale('log')
    ax.set_title( year )
    if i == 0:
        legend_elements = [
            matplotlib.lines.Line2D([0], [0], marker='o', color='w', label="10m",  markerfacecolor='grey', markersize = np.sqrt( np.sqrt(10_000_000)/50 ) ),
            matplotlib.lines.Line2D([0], [0], marker='o', color='w', label="100m", markerfacecolor='grey', markersize = np.sqrt( np.sqrt(100_000_000)/50) ),
            matplotlib.lines.Line2D([0], [0], marker='o', color='w', label="1b",   markerfacecolor='grey', markersize = np.sqrt( np.sqrt(1_000_000_000)/50) ),
        ]
        ax.legend( handles = legend_elements, title = "Population",  loc = 'lower right' )
        
    if i == 1:
        leg = ax.legend( loc = 'lower right' )
        for h in leg.legend_handles:
            h.set_sizes([100.])
plt.show()

# Time series
- Univariate EDA
- Autocorrelation
- Stationarity
- Seasonality, trend

## Time series plots; auto-correlation

In [ ]:
x = stocks['Google']['Close'].copy()
x.index = [ u.date() for u in x.index ]
x.index = pd.to_datetime( x.index )
x.index.name = 'Date'
display( np.round( x.reset_index(), 2 ).head(10) )
scale = .35
fig, ax = plt.subplots( figsize = (scale*16,scale*9), layout = 'constrained', dpi = 100 )
ax.plot(x)
ax.set_yscale('log')
ax.set_xlabel("Date (year)")
ax.set_ylabel("Price")
axis_year(ax, '%y')
remove_scientific_notation_from_vertical_axis(ax)
plt.show()

In [ ]:
if False: 
  for i in data()['dataset_id'][:100]:
    x = data(i)
    if x.shape[1] == 2 and x.columns[0] in ['time', 'Time', 'date', 'Date']: 
        x = x.set_index( x.columns[0] )[ x.columns[1] ]
        fig, ax = plt.subplots( figsize = (8,3), layout = 'constrained' )
        ax.plot(x)
        ax.set_ylabel( x.name )
        ax.set_xlabel( x.index.name )
        plt.show()
    

In [ ]:
for label in ['AirPassengers', 'sunspots', 'UKDriverDeaths', 'Nile']: 
    x = data(label)
    x = x.set_index( x.columns[0])[ x.columns[1] ]
    fig, ax = plt.subplots( figsize = (8,3), layout = 'constrained', dpi=DPI )
    ax.plot(x)
    ax.set_ylabel( x.name )
    ax.set_xlabel( x.index.name )
    if label == 'AirPassengers':
        ax.set_yscale('log')
        remove_scientific_notation_from_vertical_axis(ax)
    plt.show()
    
    a = statsmodels.tsa.stattools.acf(x, nlags = 11*12*3//2 if label == 'sunspots' else 36)
    fig, ax = plt.subplots( figsize = (15 if label == 'sunspots' else 5,2), dpi = DPI )
    b = ax.bar( np.arange(len(a)), a )
    if label in [ 'AirPassengers', 'UKDriverDeaths' ]:
        for i in [12,24,36]:
            b[i].set_facecolor('tab:red')
            #pass
        ax.set_xticks( [0, 6, 12, 18, 24, 30, 36] )
        ax.set_xticklabels( [0, None, 12, None, 24, None, 36] )
    if label == 'sunspots':
        ax.set_xticks( np.arange(0, 200, 12) )
    ax.set_ylabel( "Autocorrelation" )
    ax.set_xlabel( 
        "Lag (years)" if label == 'Nile'
        else "Lag (months)"
    )
    plt.show()



In [ ]:
d = data('AirPassengers').copy()
d = pd.concat( [ 
    d,
    pd.DataFrame( { 
        'time': 1961 + np.linspace(0,1,13)[:-1],
        'AirPassengers': np.nan
    } ),
] )
d = d.reset_index(drop=True)
d['year'] = np.floor( d['time'] ).astype(int)
d['month'] = np.round( ( d['time'] - d['year'] ) * 12  ).astype(int)+ 1
d['date'] = [ f"{u}-{v:02d}-20" for u, v in zip( d['year'], d['month'] ) ]
d['date'] = pd.to_datetime( d['date'] ) + datetime.timedelta( days = 20 )
d['date'] = [ f"{str(u)[:7]}-01" for u in d['date'] ]
d['date'] = pd.to_datetime( d['date'] ) - datetime.timedelta( days = 1 )
d = d[['date', 'AirPassengers']].copy()
d.columns = ['date', 'current']
d['previous'] = d['current'].shift()
d.iloc[:,[0,2,1]].head(10)

## Suspicious values

In [ ]:
n = 500
x = np.linspace( 0, 8*math.pi, n )
y = np.sin(x) + .5 * np.random.normal(size=n)
i = range( n//2, n//2 + 20 )
y[i] = np.nan
i = range( n//2-1, n//2 + 20 )
y = pd.Series(y).ffill().values
for highlight in [False, True]: 
    fig, ax = plt.subplots( figsize = (8,4), layout = 'constrained', dpi = 100 )
    ax.scatter(x,y, alpha = .8)
    if highlight: 
        ax.scatter(x[i], y[i])
        ax.plot( x, y, color = 'grey', linewidth = 1, zorder = -1 )        
    ax.axis('off')
    plt.show()

In [ ]:
n = 500
x = np.linspace( 0, 8*math.pi, n )
y = np.sin(x) + .2 * np.random.normal(size=n)
i = 100
y[i] = .8
y = pd.Series(y).ffill().values
for highlight in [False, True]: 
    fig, ax = plt.subplots( figsize = (8,4), layout = 'constrained', dpi = 100 )
    ax.scatter(x,y, alpha = .8)
    if highlight: 
        ax.scatter(x[i], y[i])
        ax.plot( x, y, color = 'grey', linewidth = 1, zorder = -1 )
    ax.axis('off')
    plt.show()

In [ ]:
n = 500
x = np.linspace( 0, 8*math.pi, n )
y = np.sin(x) 
i = 300
y[i:] = np.cos(x[i:]) 
y += .2 * np.random.normal(size=n)
y = pd.Series(y).ffill().values
for highlight in [False, True]: 
    fig, ax = plt.subplots( figsize = (8,4), layout = 'constrained', dpi = 100 )
    ax.scatter(x,y, alpha = .8)
    if highlight: 
        ax.scatter(x[i],   y[i],   color = 'tab:orange')
        ax.scatter(x[i-1], y[i-1], color = 'tab:orange')
        ax.plot( x, y, color = 'grey', linewidth = 1, zorder = -1 )
    ax.axis('off')
    plt.show()

## Stationarity

In [ ]:
# Stationary time series, with a histogram of half the values

n = 500
x0 = np.random.normal( size = n )
x1 = np.cumsum(x0)
x2 = x0.copy()
x2[:n//2] *= 2
for x in [x0, x1, x2]:
    
    ylim = min(x), max(x)
    bins = np.linspace( min(x), max(x), 20 )
    
    i1 = range(0, n//2)
    i2 = range(n//2, n)
    for i1,i2 in [ (range(n),[]), (i1,i2), (i2,i1) ]:
        fig = plt.figure( figsize = (8,4), layout = 'constrained', dpi=100 )
        g = GridSpec(1, 2, figure=fig, width_ratios = [5,1] )
        
        ax = plt.subplot( g[0,0] )
        ax.scatter( i1, x[i1], alpha = .8, color = 'tab:blue' )
        ax.scatter( i2, x[i2], alpha = .8, color = 'lightgrey' )

        ax.plot( range(n), x, color = 'grey', linewidth = 1, zorder = -1, alpha = .2 )

        ax.axis('off')
        
        ax = plt.subplot( g[0,1] )
        ax.hist(
            x[i1],
            bins        = bins,
            edgecolor   = "white",
            orientation = "horizontal",
        )
        ax.set_ylim( *ylim )
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.show()

## Auto-correlation

In [ ]:
# Plot a time series and x[t+1]~x[t] side by side
# First, Gaussian noise; then, real data

n = 500
np.random.seed(0)
x1 = np.random.normal(size=n)
x2 = np.cumsum(x1) + 5 * np.random.normal(size=n)

xs = [ data(label).iloc[:,1].values for label in ['AirPassengers', 'sunspots', 'UKDriverDeaths', 'Nile'] ]
x3 = data('AirPassengers')
x3 = x3.set_index( x3.columns[0])[ x3.columns[1] ].values

x = yf.Ticker("^GSPC").history(period="100y")['Close']
returns = np.log(x).diff().dropna().values

for x in [returns, np.abs(returns), x1, x2] + xs:
    n = len(x)
    x = x - np.mean(x)
    fig = plt.figure( figsize = (9,3), layout = 'constrained', dpi=100 )
    g = GridSpec(1, 2, figure=fig, width_ratios = [2,1] )
    ax = plt.subplot( g[0,0] )
    ax.scatter( range(n), x, alpha = .8 )
    #ax.scatter( range(n), x, alpha = .1 )
    ax.plot( x, color = 'lightgrey', linewidth = 1, zorder = -1 )
    ax.axis('off')
    #ax.set_ylim(-3,3)
    ax = plt.subplot( g[0,1] )
    ax.scatter( x[:-1], x[1:], alpha = .5 )
    #ax.scatter( x[:-1], x[1:], alpha = .1 )
    #ax.set_xlim(-3,3)
    #ax.set_ylim(-3,3)
    ax.axis('off')
    ax.axhline(0, linewidth = 1, color = 'black', zorder=-1)
    ax.axvline(0, linewidth = 1, color = 'black', zorder=-1)
    message = f"Cor={np.corrcoef( x[:-1], x[1:] )[0,1]:.2f}"
    t = ax.text(0.98, .98, message, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)
    t.set_path_effects( [path_effects.Stroke(linewidth=3, foreground="white"), path_effects.Normal()] )
    plt.show()

# Panel Data

# Bivariate
- Univariate EDA for each variable
- Numbers: univariate ones + correlation; missingness patterns? anscombe / datasaurus
- Scatterplot, hexbin (+log), regression line (+interval), lowess
- Clustering (k-means, hdbscan)
- density estimation, copula, median per quantile

In [ ]:
d = data('quakes').iloc[:,:2].iloc[:,::-1]

In [ ]:
d = ames[['LotArea', 'SalePrice']].copy()
d.columns = [ 'Lot Area (sq.ft.)', 'Sale Price (USD)' ]
i1 = d.iloc[:,0] <= 20_000
i2 = d.iloc[:,1] <= 400_000
d = d[i1&i2].copy()

In [ ]:
d

In [ ]:
x = d.iloc[:,0]
y = d.iloc[:,1]
sns.regplot(
  x=x,y=y,
  lowess=True, 
  scatter_kws={'alpha': .2}, 
  line_kws={'color':'orange', 'linewidth': 5},
)

## Correlation

In [ ]:
d.corr().iloc[0,1]

In [ ]:
{ 
    'Correlation': np.corrcoef(d.T)[0,1],
}  

In [ ]:
# Explain what correlation is
np.random.seed(0)
N = 500
X1 = np.random.normal(size=N)
X2 = np.random.normal(size=N)
rhos = [-.999, -.9, -.5, 0, .5, .9, .999]
fig, axs = plt.subplots(1, len(rhos), figsize = (12,2), layout = "constrained", dpi=100)
for i, rho in enumerate(rhos):
    ax = axs.flatten()[i]
    Y = rho * X1 + np.sqrt( 1 - rho ** 2 ) * X2
    ax.scatter( X1, Y, alpha = .3 )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title( f"ρ={rho:.1f}" )
plt.show()

In [ ]:
# Linear and non-linear relations
n = 1000
for which in [False, True]:
    np.random.seed(0)
    fig, axs = plt.subplots(1,2, figsize = (8,3.5), layout = 'constrained', dpi = 100)
    
    ax = axs[0]
    x = np.random.normal( size = n )
    y = np.random.normal( size = n ) + x
    xs = np.linspace( x.min(), x.max() )
    ax.scatter( x, y, alpha = .5 )
    if which:
        ax.plot( xs, xs, color = 'orange', linewidth = 10 )
    
    ax = axs[1]
    x = np.random.uniform( -2, 3, size=n )
    y = np.random.normal( size = n ) * .2 + np.sin(x)/x
    xs = np.linspace( x.min(), x.max() )
    ys = np.sin(xs) / xs
    ax.scatter( x, y, alpha = .5 )
    if which: 
        ax.plot( xs, ys, color = 'orange', linewidth = 10 )
        
    for ax in axs: 
        ax.axis('off')
    plt.show()

In [ ]:
# Heteroskedasticity
N = 1000
xs = np.linspace( -1.5, 3, N )
ys = np.sin(xs)
z0 = ( .2 + np.linspace(0,1,N) )
z1 = .4 * np.random.normal(size=N)
z2 = z0 * np.random.normal(size=N)
for which in [False, True]:
    fig, axs = plt.subplots( 1, 2, figsize = (8,3.5), layout = 'constrained', dpi=100)
    ax = axs[0]
    ax.scatter( xs, ys + z1, alpha = .5)
    a = .8
    if which: 
        ax.fill_between( xs, ys - a, ys + a, alpha = .5, zorder = -1 )
    ax = axs[1]
    ax.scatter( xs, ys + z2, alpha = .5)
    if which: 
        ax.fill_between( xs, ys - 2*z0, ys + 2*z0, alpha = .5, zorder = -1 )
    for ax in axs:
        ax.axis('off')
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
# Heteroskedasticity
N = 1000
xs = np.linspace( - 2 * math.pi, 2 * math.pi, N )
ys = np.sin(xs)
ys += ( .2 + np.linspace(0,1,N) ) * np.random.normal(size=N)
fig, ax = plt.subplots( figsize = (8,4.5), layout = 'constrained', dpi=100)
ax.scatter( xs, ys, alpha = .5)
ax.axis('off')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

## Scatterplots; density; clustering

In [ ]:
fig, ax = plt.subplots( figsize = (4,4), layout='constrained', dpi = DPI )
ax.scatter( d.iloc[:,0], d.iloc[:,1] )
ax.set_xlabel( d.columns[0] )
ax.set_ylabel( d.columns[1] )
if d.columns[0]  == 'long': 
    ax.set_aspect(1)
plt.show()

fig, ax = plt.subplots( figsize = (4,4), layout='constrained', dpi = DPI )
ax.scatter( d.iloc[:,0], d.iloc[:,1], alpha = .1, s = 100 )
ax.set_xlabel( d.columns[0] )
ax.set_ylabel( d.columns[1] )
if d.columns[0]  == 'long': 
    ax.set_aspect(1)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi=DPI )
ax.hexbin(d.iloc[:,0], d.iloc[:,1], cmap = 'Blues', gridsize = 20, bins = 'log' )
ax.set_xlabel( d.columns[0] )
ax.set_ylabel( d.columns[1] )
if d.columns[0]  == 'long': 
    ax.set_aspect(1)
plt.show()

In [ ]:
x, y = d.values.T
xx, yy = np.mgrid[ 
    min(x) : max(x) : 100j, 
    min(y) : max(y) : 100j, 
]
positions = np.vstack([xx.ravel(), yy.ravel()])
kernel = scipy.stats.gaussian_kde( 
    np.vstack([x, y]),
    bw_method = .2,
)
f = np.reshape(kernel(positions).T, xx.shape)

fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = DPI)
ax.set_xlim( min(x), max(x) )
ax.set_ylim( min(y), max(y) )
ax.contourf(xx, yy, f, cmap='Blues')
#ax.imshow(np.rot90(f), cmap='Blues', extent=[xmin, xmax, ymin, ymax])
ax.contour(xx, yy, f, colors='k')
#ax.clabel(cset, inline=1, fontsize=10)
#ax.axes.xaxis.set_visible(False)
#ax.axes.yaxis.set_visible(False)
ax.set_xlabel( d.columns[0] )
ax.set_ylabel( d.columns[1] )
if d.columns[0]  == 'long': 
    ax.set_aspect(1)
plt.show()

In [ ]:
d = load_penguins().iloc[:,[2,3]].dropna().copy()

In [ ]:
DPI = 100
for model in [ 
    sklearn.cluster.DBSCAN(),
    sklearn.cluster.KMeans(n_clusters=2),
]:
    c = model.fit_predict(d/1.7)
    color = [ matplotlib.cm.Set1.colors[i] for i in c ]
    color = 'tab:blue'
    c = 0 * np.array(c)
    fig, ax = plt.subplots( figsize = (4,4), layout='constrained', dpi=DPI )
    ax.scatter( d.iloc[:,0], d.iloc[:,1], color = color, s = [ 100 if i >= 0 else 30 for i in c ], alpha = .5 )
    ax.set_xlabel( d.columns[0] )
    ax.set_ylabel( d.columns[1] )
    if d.columns[0]  == 'long': 
        ax.set_aspect(1)
    plt.show()

## Fitting a line, a curve

In [ ]:
x, y = d.iloc[:,0], d.iloc[:,1]

model = sklearn.linear_model.LinearRegression()
model.fit( pd.DataFrame(x.values), y )
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi=DPI )
ax.scatter( x, y )
ax.axline( (x.mean(), y.mean()), slope = model.coef_[0], color = 'orange', linewidth = 5 )
if d.columns[0]  == 'long': 
    ax.set_aspect(1)
    ax.set_xlabel( "Longitude" )
    ax.set_ylabel( "Latitude" )
else:
    ax.set_xlabel( d.columns[0] )
    ax.set_ylabel( d.columns[1] )    
plt.show()

ys = statsmodels.nonparametric.smoothers_lowess.lowess(y, x, frac=.5)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi=DPI )
ax.scatter( x, y )
ax.plot( ys[:,0], ys[:,1], color = 'orange', linewidth = 5 )
if d.columns[0]  == 'long': 
    ax.set_aspect(1)
    ax.set_xlabel( "Longitude" )
    ax.set_ylabel( "Latitude" )
else:
    ax.set_xlabel( d.columns[0] )
    ax.set_ylabel( d.columns[1] )
plt.show()

In [ ]:
X, y = sklearn.datasets.load_diabetes(return_X_y=True, as_frame=True)  # The data has been pre-processed: the qualitative variables have been replaced by avg(y)...
x = X['s5']
#print( sklearn.datasets.load_diabetes()['DESCR'] )

In [ ]:
fig, ax = plt.subplots( figsize = (5,4), layout = 'constrained' )
ax.scatter( x, y )
ax.set_xlabel( "log(triglycerides)" )
ax.set_ylabel( "disease progression" )
plt.show()

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit( pd.DataFrame(x.values), y )
xs = np.linspace( min(x), max(x), 100 )
ys = model.predict( pd.DataFrame( xs ) )
fig, ax = plt.subplots( figsize = (5,4), layout = 'constrained' )
ax.scatter( x, y )
ax.plot( xs, ys, color = 'orange', linewidth = 5 )
ax.set_xlabel( "log(triglycerides)" )
ax.set_ylabel( "disease progression" )
plt.show()

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit( pd.DataFrame(x.values), y )
fig, ax = plt.subplots( figsize = (5,4), layout = 'constrained' )
ax.scatter( x, y )
ax.axline( (x.mean(), y.mean()), slope = model.coef_[0], color = 'orange', linewidth = 5 )
ax.set_xlabel( "log(triglycerides)" )
ax.set_ylabel( "disease progression" )
plt.show()

In [ ]:
ys = statsmodels.nonparametric.smoothers_lowess.lowess(y, x, frac=.5)
fig, ax = plt.subplots( figsize = (5,4), layout = 'constrained' )
ax.scatter( x, y )
ax.plot( ys[:,0], ys[:,1], color = 'orange', linewidth = 5 )
ax.set_xlabel( "log(triglycerides)" )
ax.set_ylabel( "disease progression" )
plt.show()

In [ ]:
sns.regplot(x=x, y=y, line_kws = {'color':'orange', 'linewidth':5})

In [ ]:
sns.regplot(x=x, y=y, lowess=True, line_kws = {'color':'orange', 'linewidth':5})

## Copula

In [ ]:
plot_copula(x,y)

In [ ]:
#sns.scatterplot( x=x, y=y )
#sns.regplot(x=x, y=y, lowess=False, ci=99)
#sns.regplot(x=x, y=y, lowess=True,  line_kws = {'color':'orange', 'linewidth':5})

## Anscombe, data-saurus

In [ ]:
d = data("anscombe")
fig, axs = plt.subplots( 2, 2, figsize = (8,4.5), layout = 'constrained', dpi=DPI )
for i in range(4): 
    ax = axs.flatten()[i]
    x = d[f"x{i+1}"]
    y = d[f"y{i+1}"]
    ax.scatter( x, y )

    message = (
        f"Mean(x) = {x.mean():.2f}\n"
        f"Mean(y) = {y.mean():.2f}\n"
        f"Std(x) = {x.std():.2f}\n"
        f"Std(y) = {y.std():.2f}\n"
        f"Cor(x,y) = {np.corrcoef(x, y)[0,1]:.2f}"
    )    
    ax.text(0.02, .98, message, horizontalalignment='left', verticalalignment='top', transform=ax.transAxes)

    model = sklearn.linear_model.LinearRegression()
    model.fit( pd.DataFrame(x.values), y )
    ax.axline( (0, model.intercept_), slope = model.coef_[0], color = 'orange', alpha = .8, linewidth = 2, zorder = -1 )    
    
    ax.set_xlim( 3, 20 )
    ax.set_ylim( 3, 13 )
plt.show()

In [ ]:
# datasaurus dozen
# https://www.research.autodesk.com/app/uploads/2023/03/same-stats-different-graphs.pdf_rec2hRjLLGgM7Cn2T.pdf
!wget -nc https://raw.githubusercontent.com/jmatejka/same-stats-different-graphs/master/samestats/datasets/generated/DatasaurusDozen.tsv

In [ ]:
d = pd.read_csv( "DatasaurusDozen.tsv", sep='\t')
names = d['dataset'].unique()[1:]
nr, nc = mfrow(len(names))
scale = .9
for text in [False, True]: 
    fig, axs = plt.subplots( nr, nc, figsize = (scale*16,scale*9), layout = 'constrained', dpi=DPI)
    for i, name in enumerate(names):
        ax = axs.flatten()[i]
        j = d['dataset'] == name
        x = d['x'][j]
        y = d['y'][j]
        ax.scatter(x, y)
        ax.set_xticks([])
        ax.set_yticks([])

        if text: 
            message = (
                f"Mean(x) = {x.mean():.2f}\n"
                f"Mean(y) = {y.mean():.2f}\n"
                f"Std(x) = {x.std():.2f}\n"
                f"Std(y) = {y.std():.2f}\n"
                f"Cor(x,y) = {np.corrcoef(x, y)[0,1]:.2f}"
            )    
            t = ax.text(0.02, .98, message, horizontalalignment='left', verticalalignment='top', transform=ax.transAxes)
            t.set_path_effects( [path_effects.Stroke(linewidth=3, foreground="white"), path_effects.Normal()] )
    
    remove_empty_axes(axs)
    plt.show()

## Bivariate outliers

In [ ]:
# Bivariate outliers
np.random.seed(1)
X = np.random.normal( size = (1000,2) )
a = 2
A = np.array( [ [1, a], [a, 1] ] )
X = X @ A
X[0,:] = [-4,5]

for which in [0,1,2]:
    fig, ax = plt.subplots( figsize = (3,3), layout = 'constrained', dpi = 100 )
    ax.scatter( X[:,0], X[:,1], alpha = .3 )
    if which > 0:
        add_ellipse( A.T @ A, [0,0], ax=ax, radius = 6, color = 'orange' )
    if which > 1:
        ax.scatter( X[0,0], X[0,1], color = 'tab:red' )
    ax.axis('off')
    plt.show()

# Multivariate

- Univariate EDA, in particular the missing values
- pairs plot, corrplot (+order from hclust)
- hclust for the variables
- dimension reduction: PCA, t-SNE, UMAP (+clustering)

In [ ]:
d = data("Electricity")

In [ ]:
#data("Electricity", show_doc=True)
new_names = {
    'cost': 'total cost',
    'q': 'total output',
    'pl': 'wage rate',
    'sl': 'cost share for labor',
    'pk': 'capital price index',
    'sk': 'cost share for capital',
    'pf': 'fuel price',
    'sf': 'cost share for fuel',
}
d.rename( new_names, axis=1, inplace=True )

In [ ]:
d.head(5)

## Numbers

In [ ]:
print( d.describe() )

## Boxplots

In [ ]:
for which in [0,1,2]:
    fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
    if which == 0:
        b = ax.boxplot(d, vert=False, patch_artist=True)
    elif which == 1:
        b = ax.boxplot(d / d.std(), vert=False, patch_artist=True)
    else: 
        ax.violinplot(d / d.std(), vert=False)
    if which in [0,1]:
        for u in b['boxes']:
            u.set_facecolor('lightblue')
            u.set_edgecolor('black')
        for u in b['medians']:
            u.set_color('black')    
    ax.set_yticks( 1 + np.arange(d.shape[1]) )
    ax.set_yticklabels( d.columns )
    plt.show()

In [ ]:
dd = d / d.std()
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
if True: 
    b = ax.boxplot(dd, vert=False, patch_artist=True)
    for u in b['boxes']:
        u.set_facecolor('None')
        u.set_edgecolor('black')
    for u in b['medians']:
        u.set_color('black')    
ax.set_yticks( 1 + np.arange(dd.shape[1]) )
ax.set_yticklabels( dd.columns )    

a = .3
for i in range(dd.shape[1]): 
    ax.scatter( dd.iloc[:,i], dd.shape[0] * [i+1] + np.random.uniform(-a,a,size=dd.shape[0]), alpha = .2 )
plt.show()

## Pair-plot

In [ ]:
plt.rcParams['figure.dpi'] = 100
sns.pairplot(d)

In [ ]:
k = d.shape[1]
fig, axs = plt.subplots(k, k, figsize=(9,9), layout='constrained')
for i, id1 in enumerate( d.columns ):
    for j, id2 in enumerate( d.columns ):
        ax = axs[i,j]
        if i == j: 
            ax.hist( d[id1], bins=20, density = True )
            ax.text(.5, .99, id1, horizontalalignment='center', verticalalignment='top', transform=ax.transAxes)
            for side in ['left', 'right', 'top']:
                ax.spines[side].set_visible(False)
            ax.set_yticks([])
        if i != j:
            ax.scatter( d[id2], d[id1], alpha = .3 )
            ax.text(.5, .99, f"{np.corrcoef( d[id1], d[id2] )[0,1]:.2f}", horizontalalignment='center', verticalalignment='top', transform=ax.transAxes)
            ys = statsmodels.nonparametric.smoothers_lowess.lowess(d[id1], d[id2], frac=.5)
            ax.plot( ys[:,0], ys[:,1], color = 'orange', linewidth = 3 )
            ax.axis('off')
plt.show()

In [ ]:
for which in [0,1,2,3,4]: 
        
    k = d.shape[1]
    fig, axs = plt.subplots(k, k, figsize=(16,9), layout='constrained', dpi=DPI)
    for i, id1 in enumerate( d.columns ):
        for j, id2 in enumerate( d.columns ):
            ax = axs[i,j]
            if i == j: 
                ax.hist( d[id1], bins=20, density = True )
                #ax.set_title( id1 )
                ax.text(.5, .99, id1, horizontalalignment='center', verticalalignment='top', transform=ax.transAxes)
                for side in ['left', 'right', 'top']:
                    ax.spines[side].set_visible(False)
                ax.set_yticks([])
            if i != j and which > 0 and ( which < 4 or i < j ):
                ax.scatter( d[id2], d[id1], alpha = .3 )
    
                # Show the correlation
                if which > 1:
                    ax.text(.5, .99, f"{np.corrcoef( d[id1], d[id2] )[0,1]:.2f}", horizontalalignment='center', verticalalignment='top', transform=ax.transAxes)
    
                # Add a regression line
                if which > 2:
                    ys = statsmodels.nonparametric.smoothers_lowess.lowess(d[id1], d[id2], frac=.5)
                    ax.plot( ys[:,0], ys[:,1], color = 'orange', linewidth = 3 )
                
                ax.axis('off')

            if i > j and which >= 4: 
                plot_copula( d[id2], d[id1], ax = ax )
                
    remove_empty_axes(axs)
    plt.show()

## Correlation matrix (heatmap)

In [ ]:
fig, ax = plt.subplots( figsize = (5,5), layout = 'constrained', dpi=100 )
corrplot( d.corr(), labels=True, ax=ax )
plt.show()

In [ ]:
sns.clustermap(d.corr(), vmin=-1, vmax=+1, cmap='RdBu', figsize=(6,6), annot=True, fmt=".2f")
#sns.clustermap?
#scipy.spatial.distance.pdist?

In [ ]:
C = d.corr()
D = np.sqrt(1-np.abs(C))
D = scipy.spatial.distance.squareform( D )
Z = scipy.cluster.hierarchy.ward(D)
sns.clustermap(
    C, 
    vmin=-1, vmax=+1, 
    cmap='RdBu', 
    figsize=(6,6), 
    annot=True, fmt=".2f",
    row_linkage = Z, col_linkage = Z,
)

In [ ]:
# TODO: Univariate numbers

## Dimension reduction 

In [ ]:
# Dimension reduction: PCA, t-SNE, UMAP (+clustering)

# First, transform the variables to make them more Gaussian
dd = d.copy()
for i in dd.columns: 
    if np.any( dd[i] <= 0 ):
        continue
    before = scipy.stats.normaltest( dd[i] ).pvalue
    after  = scipy.stats.normaltest( np.log(dd[i]) ).pvalue
    if before < after:
        dd[i] = np.log(dd[i])
        #print( f"Taking the log of {i}: {before} < {after}" )
    else: 
        pass
        #print( f"Keeping {i}" )

X = dd.copy()
X -= X.mean()
X /= X.std()
u, s, vh = np.linalg.svd(X)
U = u[:,:2]

for X in [
    sklearn.decomposition.PCA(2).fit_transform(dd),
    U,
    UMAP(random_state=0).fit_transform(dd),

    sklearn.decomposition.PCA(2).fit_transform( uniformize(d) ),
    UMAP(random_state=0).fit_transform( uniformize(d) ),
    
]:
    nr, nc = mfrow( d.shape[1] )
    #fig, axs = plt.subplots( nr, nc, figsize = (8,4.5), layout = 'constrained', dpi=DPI )
    fig, axs = plt.subplots( nr, nc, figsize = (5.5,5.5), layout = 'constrained', dpi=DPI )
    for i, column in enumerate( d.columns ): 
        ax = axs.flatten()[i]
        ax.scatter(X[:,0], X[:,1], c = uniformize(d[column]), s = 60)
        ax.axis('off')
        ax.set_title(column)
    remove_empty_axes(axs)
    plt.show()    

In [ ]:
X = dd.copy()
X -= X.mean()
X /= X.std()
u, s, vh = np.linalg.svd(X)
vh /= 3.5
for variables in [False, True]:
    fig, ax = plt.subplots( figsize = (4,4), dpi = DPI)
    ax.scatter( u[:,0], u[:,1], alpha = .5, s = 100 )
    ax.axhline( 0, linestyle = ":", color = 'black', linewidth = 1 )
    ax.axvline( 0, linestyle = ":", color = 'black', linewidth = 1 )
    if variables: 
      for i in range(X.shape[1]):
        if np.abs( vh[:2,i] ).sum() > .01:
            ax.arrow( 
                0, 0, vh[0,i], vh[1,i], 
                length_includes_head = True, 
                width = 0.005,
                head_width = None, # 3*width
                head_length = None, # 1.5*head_width
                overhang = .8,
                color = 'tab:red',
            )
        t = ax.text( vh[0,i], vh[1,i], X.columns[i], va='center', ha='center' )
        t.set_path_effects( [path_effects.Stroke(linewidth=3, foreground="white", alpha = .5), path_effects.Normal()] )
    ax.axis('off')
    plt.show()

In [ ]:
X = dd.copy()
X -= X.mean()
X /= X.std()
u, s, vh = np.linalg.svd(X)
vh /= 3.5
for i, column in enumerate( d.columns ): 
    for add_arrow in [ False, True ]:
        fig, ax = plt.subplots( figsize = (4,4), dpi = DPI)
        ax.scatter( u[:,0], u[:,1], c = uniformize(d[column]), s = 100)
        ax.axhline( 0, linestyle = ":", color = 'black', linewidth = 1 )
        ax.axvline( 0, linestyle = ":", color = 'black', linewidth = 1 )    
        if add_arrow:
            if np.abs( vh[:2,i] ).sum() > .01:
                ax.arrow( 
                    0, 0, vh[0,i], vh[1,i], 
                    length_includes_head = True, 
                    width = 0.005,
                    head_width = None, # 3*width
                    head_length = None, # 1.5*head_width
                    overhang = .8,
                    color = 'tab:red',
                )
            t = ax.text( vh[0,i], vh[1,i], X.columns[i], va='center', ha='center' )
            t.set_path_effects( [path_effects.Stroke(linewidth=3, foreground="white", alpha = .5), path_effects.Normal()] )        
        ax.axis('off')
        #ax.set_title(column)
        plt.show()    

In [ ]:
# Clustering
SEED = 0
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
X = UMAP(random_state=SEED).fit_transform(dd)
model = sklearn.cluster.DBSCAN( eps = .7, min_samples = 5 )
c = model.fit(X).labels_
for cluster in [False, True]:
    fig, ax = plt.subplots( figsize = (4.5,4.5), layout = 'constrained', dpi = DPI )
    if cluster: 
        ax.scatter( X[:,0], X[:,1], c=c, cmap = 'Set1', s = 100, alpha = .8 )
    else: 
        ax.scatter( X[:,0], X[:,1], s = 100, alpha = .8 )
    ax.axis('off')
    plt.show()

# Text

# Prerequisites

In [ ]:
xs = np.linspace( -2, 2, 100 )
ys = xs ** 2
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( [-2,-1,1,2], [0,0,0,0], marker = '+', color = 'black' )
ax.scatter( [0,0,0,0], [1,2,3,4], marker = '+', color = 'black' )
ax.axis('off')
plt.show()

In [ ]:
a = 4
xs = np.linspace( -a, a, 101 )
ys = 1 / xs
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-a,a+1), (2*a+1)*[0], marker = '+', color = 'black' )
ax.scatter( (2*a+1)*[0], np.arange(-a,a+1), marker = '+', color = 'black' )
ax.set_xlim( -1.02*a, 1.02*a )
ax.set_ylim( -1.02*a, 1.02*a )
ax.axis('off')
plt.show()

In [ ]:
a = 4
xs = np.linspace( -a, a, 101 )
ys = 2*xs - 1
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-a,a+1), (2*a+1)*[0], marker = '+', color = 'black' )
ax.scatter( (2*a+1)*[0], np.arange(-a,a+1), marker = '+', color = 'black' )
ax.set_xlim( -1.02*a, 1.02*a )
ax.set_ylim( -1.02*a, 1.02*a )
ax.axis('off')
plt.show()

In [ ]:
a = 4
xs = np.linspace( -a, a, 101 )
ys = xs ** 3
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-a,a+1), (2*a+1)*[0], marker = '+', color = 'black' )
ax.scatter( (2*a+1)*[0], np.arange(-a,a+1), marker = '+', color = 'black' )
ax.set_xlim( -1.02*a, 1.02*a )
ax.set_ylim( -1.02*a, 1.02*a )
ax.axis('off')
plt.show()

In [ ]:
a = 4
xs = np.linspace( -a, a, 101 )
ys = np.sin(xs)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-a,a+1), (2*a+1)*[0], marker = '+', color = 'black' )
ax.scatter( (2*a+1)*[0], np.arange(-a,a+1), marker = '+', color = 'black' )
ax.set_xlim( -1.02*a, 1.02*a )
ax.set_ylim( -1.02*a, 1.02*a )
ax.axis('off')
plt.show()

In [ ]:
a = 4
xs = np.linspace( -a, a, 101 )
ys = np.cos(xs)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-a,a+1), (2*a+1)*[0], marker = '+', color = 'black' )
ax.scatter( (2*a+1)*[0], np.arange(-a,a+1), marker = '+', color = 'black' )
ax.set_xlim( -1.02*a, 1.02*a )
ax.set_ylim( -1.02*a, 1.02*a )
ax.axis('off')
plt.show()

In [ ]:
xs = np.linspace( 0, 4, 100 )
ys = np.sqrt(xs)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( [1,2,3,4], [0,0,0,0], marker = '+', color = 'black' )
ax.scatter( [0,0,0,0], [-1,-2,1,2], marker = '+', color = 'black' )
#ax.plot( [.2,1.8], [-.8,.8], color = 'black', zorder = 11 )
ax.axis('off')
ax.set_xlim( -.2, 4.2 )
ax.set_ylim( -.2, 2.2 )
ax.set_aspect(1)
plt.show()

In [ ]:
xs = np.linspace( -4, 4, 100 )
ys = np.exp(xs)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-5,6), np.zeros(11), marker = '+', color = 'black' )
ax.scatter( [0,0,0,0], [-1,-2,1,2], marker = '+', color = 'black' )
#ax.plot( [.2,1.8], [-.8,.8], color = 'black', zorder = 11 )
ax.axis('off')
ax.set_xlim( -2.2, 2.2 )
ax.set_ylim( -.2, 2.2 )
ax.set_aspect(1)
plt.show()

In [ ]:
xs = np.linspace( 0, 4, 100 )
ys = np.log(xs)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( [1,2,3,4], [0,0,0,0], marker = '+', color = 'black' )
ax.scatter( [0,0,0,0], [-1,-2,-3,1], marker = '+', color = 'black' )
#ax.plot( [.2,1.8], [-.8,.8], color = 'black', zorder = 11 )
ax.axis('off')
plt.show()

In [ ]:
xs = np.linspace( -4, 4, 1000 )
ys = np.log(xs)
zs = np.exp(xs)
fig, ax = plt.subplots( figsize = (4,4), layout = 'constrained', dpi = 100 )
ax.plot( xs, ys, linewidth = 5, zorder = 10 )
ax.plot( xs, zs, linewidth = 5, zorder = 10 )
ax.plot( xs, xs, linewidth = 3, zorder = 10, color = 'black' )
ax.axhline( 0, color = 'black' )
ax.axvline( 0, color = 'black' )
ax.scatter( np.arange(-4,5), 9*[0], marker = '+', color = 'black' )
ax.scatter( 9*[0], np.arange(-4,5), marker = '+', color = 'black' )
#ax.plot( [.2,1.8], [-.8,.8], color = 'black', zorder = 11 )
ax.set_xlim( -4.1, 4.1 )
ax.set_ylim( -4.1, 4.1 )
ax.axis('off')
plt.show()

In [ ]:
import yfinance as yf
x = yf.Ticker("^GSPC").history(period="100y")['Close']

In [ ]:
for log in [False, True]:
    fig, ax = plt.subplots( figsize = (10,4.5), layout = 'constrained', dpi = DPI)
    ax.plot(x)
    if log: 
        ax.set_yscale('log')
        remove_scientific_notation_from_vertical_axis(ax)
    axis_decade(ax)
    ax.set_xlabel( "Date" )
    ax.set_ylabel( "Price" )
    plt.show()

In [ ]:
# Logarithmic scale: sizes

sizes = """
object,size
proton,1.67e-15
neutrino,2e-12
hydrogen,106e-12
DNA strand diameter,2e-9
virus,50e-9
bacteria,3e-6
cell,10e-6
insects,1e-3
birds,10e-3
humans,1.65
buildings,10
skyscrapers,100
Burj Khalifa,828
Everest,8849
City (Tokyo),160_000
Country (Spain), 875_000
Earth,12_700_000
Jupiter,140_000_000
Sun,1_400_000_000
our black hole,24_000_000_000
solar system,36_000_000_000_000
distance to alpha centauri,40_000_000_000_000_000
"""
sizes = pd.read_csv( StringIO(sizes.replace("_","")) )

In [ ]:
fig, ax = plt.subplots( figsize = (32/2,4), layout = 'constrained', dpi = 300 )
for k in range(-15,17):
    ax.scatter( [k], 0, marker = '+', s = 100, color = 'black' )
    label = "$10^{" f"{k}" "}$"
    if k == 0: 
        label = "1m"
    ax.text( 
        k - .2, -.015, label, 
        ha = 'left', va = 'center', 
        #family = 'Bitter', weight = 'semibold', 
        size = 12,
    )
ax.plot( [-15.5, 16.9], [0,0], color = 'black' )
for _, row in sizes.iterrows():
    ax.text( 
        np.log10( row['size'] ), .01,
        row['object'], 
        rotation = 90, ha = 'center', va = 'bottom', 
        family = 'Bitter', weight = 'semibold', size = 18,
    )
ax.axis('off')
ax.set_xlim( -15.5, 16.8 )
plt.show()

In [ ]:
# Gaussian distribution
N = 1_000_000
np.random.seed(1)
x = np.random.normal( size = N )
x = x[ np.abs(x) < 5 ]

for density in [0,1,2]:
    fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained', dpi = 100)
    if density <= 1:
        ax.hist( x, bins = np.linspace(-5,5,21), facecolor='lightblue', edgecolor='tab:blue', density=True)
    if density >= 1: 
        xs = np.linspace( -5, 5, 500 )
        ys = np.exp( - xs ** 2 / 2 ) / math.sqrt( 2 * math.pi ) 
        ax.plot( xs, ys, linewidth = 5, color = 'tab:orange' )
    ax.axhline( 0, linewidth = 1, color = 'black', alpha = .5, zorder = -2)
    ax.set_ylim( -.01,.45 )
    ax.set_xlim( -5, 5 )
    ax.axis('off')
    plt.show()

In [ ]:
fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained', dpi = 100 )
d = data('quakes').iloc[:,::-1].values
ax.scatter( d[:,0], d[:,1] + np.random.normal(size=d.shape[0]) * .05, s = 10, alpha = .5 )
ax.axis('off')
plt.show()

fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained', dpi = 100 )
np.random.seed(1)
x = np.random.normal( size = 100 )
x = np.cumsum(x)
ax.plot(x, linewidth = 5)
ax.axis('off')
plt.show()

In [ ]:
# The 3 main types of plots (matplotlib)
fig, axs = plt.subplots( 1, 3, figsize = (9, 3), layout = 'constrained', dpi = 100 )

ax = axs[0]
N = 1_000_000
np.random.seed(1)
x = np.random.normal( size = N )
x = x[ np.abs(x) < 5 ]
ax.hist( x, bins = np.linspace(-5,5,21), facecolor='lightblue', edgecolor='tab:blue', density=True)
ax.axis('off')

ax = axs[1]
d = data('quakes').iloc[:,::-1].values
ax.scatter( d[:,0], d[:,1] + np.random.normal(size=d.shape[0]) * .05, s = 10, alpha = .5 )
ax.axis('off')

ax = axs[2]
np.random.seed(1)
x = np.random.normal( size = 100 )
x = np.cumsum(x)
ax.plot(x, linewidth = 5)
ax.axis('off')

plt.show()

In [ ]:
# The 3 main types of plots (seaborn)
fig, axs = plt.subplots( 1, 3, figsize = (9, 3), layout = 'constrained', dpi = 300 )

ax = axs[0]
N = 1_000_000
np.random.seed(1)
x = np.random.normal( size = N )
x = x[ np.abs(x) < 5 ]
#ax.hist( x, bins = np.linspace(-5,5,21), facecolor='lightblue', edgecolor='tab:blue', density=True)
sns.histplot( x, ax=ax, bins = np.linspace(-5,5,21), facecolor='lightblue', edgecolor='tab:blue' ) # , density=True)
ax.axis('off')

ax = axs[1]
d = data('quakes').iloc[:,::-1].values
#ax.scatter( d[:,0], d[:,1] + np.random.normal(size=d.shape[0]) * .05, s = 10, alpha = .5 )
d = data('quakes').iloc[:,::-1]
d.iloc[:,1] += np.random.normal(size=d.shape[0]) * .05
sns.scatterplot( d, x=d.columns[0], y=d.columns[1], ax = ax, s = 20, alpha = .5 )
ax.axis('off')

ax = axs[2]
np.random.seed(1)
x = np.random.normal( size = 100 )
x = np.cumsum(x)
x = pd.Series(x)
#ax.plot(x, linewidth = 5)
sns.lineplot(x, ax=ax, linewidth = 5)
ax.axis('off')

plt.show()

In [ ]:
# Plot for the HackerRank test
if False:
        
    x0 = np.random.normal( size = 10_000 )
    x1 = scipy.stats.skewnorm.rvs(a=3, loc=-1, size=10_000)
    x2 = scipy.stats.skewnorm.rvs(a=-3, loc=+1, size=10_000)
    x3 = data("faithful").iloc[:,1]
    
    for x in [x0,x1,x2,x3]:
        fig, ax = plt.subplots( figsize = (4,3), layout = 'constrained', dpi = 100 )
        #ax.hist( x, bins = np.linspace(-5,5,21), facecolor='lightblue', edgecolor='tab:blue', density=True)
        ax.hist( x, bins = 20, facecolor='lightblue', edgecolor='tab:blue', density=True)
        ax.axis('off')
        plt.show()

In [ ]:
# Explain what correlation is
np.random.seed(0)
N = 500
X1 = np.random.normal(size=N)
X2 = np.random.normal(size=N)
rhos = [-.999, -.93, -.5, 0, .5, .93, .999]
for i, rho in enumerate(rhos):
    fig, ax = plt.subplots( figsize = (3,3), layout = 'constrained', dpi = 100 )
    Y = rho * X1 + np.sqrt( 1 - rho ** 2 ) * X2
    ax.scatter( X1, Y, alpha = .3 )
    ax.set_xticks([])
    ax.set_yticks([])
    #ax.set_title( f"ρ={rho:.1f}" )
    ax.axis('off')
    plt.show()

# Plotting advice

## TODO Label the axes

## Prefer line plots

In [ ]:
d = pd.DataFrame( 
    {
        'Small': [35, 34, 4, 45, 12, 22, 11, -3, -19, 44, 26, 8, 19, 20, 9, -4, 32, 18, 1],
        'Large': [18, 4, -12, 30, 8, 13, 12, -2, -35, 35, 23, 7, 12, 16, 11, -6, 20, 16, -5],
    }, 
    #index = pd.date_range( "2000-01-01", freq="Y", periods = 19 ),
    index = np.arange(2000,2019),
)

fig, axs = plt.subplots( 1, 2, figsize = (8,3.5), layout = 'constrained', dpi = 100 )
ax = axs[0]
d.plot(kind='bar', ax=ax, width = .8)
ax.set_ylabel( "Average return" )

ax = axs[1]
ax.plot( d.index, d['Small'], linewidth = 3, label = 'Small' )
ax.plot( d.index, d['Large'], linewidth = 3, label = 'Large' )
ax.axhline( 0, linewidth = 1, color = 'black', linestyle = ':' )
leg = ax.legend()
for i in leg.legend_handles:
    i.set_linewidth(7)
    i.set_solid_capstyle('butt')
    
for ax in [ax]: 
    ax.set_xlabel( "Year" )
    ax.set_ylabel( "Average return" )
    ax.set_xticks( np.arange(2000,2019,5) )
    ax.set_xticks( np.arange(2000,2019,1), minor = True )
plt.show()

In [ ]:
pd.date_range( "2000-01-01", freq="Y", periods = 19 )

## TODO Date format

## Years are intervals

In [ ]:
x = yf.Ticker("^GSPC").history(period="max")['Close']

In [ ]:
y = x.tail(20_000)
fig, axs = plt.subplots( 1, 2, figsize = (10,4.5), layout = 'constrained', dpi = 100)
for ax in axs:
    ax.set_yscale('log')
    ax.set_xlabel( "Date" )
    ax.set_ylabel( "Price" )
    #ax.set_xlim( pd.to_datetime('2000-01-01'), pd.to_datetime('2015-01-01') )
ax = axs[0]
ax.plot(y)
ax = axs[1]
ax.plot(y)
remove_scientific_notation_from_vertical_axis(ax)
axis_decade(ax)
plt.show()

In [ ]:
y = x.tail(2_000)
fig, axs = plt.subplots( 1, 2, figsize = (10,4.5), layout = 'constrained', dpi = 100)
for ax in axs:
    ax.set_yscale('log')
    ax.set_xlabel( "Date" )
    ax.set_ylabel( "Price" )
    #ax.set_xlim( pd.to_datetime('2000-01-01'), pd.to_datetime('2015-01-01') )
ax = axs[0]
ax.plot(y)
ax = axs[1]
ax.plot(y)
remove_scientific_notation_from_vertical_axis(ax)
axis_year(ax,'%Y')
plt.show()

## Log scale

In [ ]:
y = x #.tail(2_000)
fig, axs = plt.subplots( 1, 2, figsize = (10,4.5), layout = 'constrained', dpi = 100)
for ax in axs:
    ax.set_xlabel( "Date" )
    ax.set_ylabel( "Price" )
    #ax.set_xlim( pd.to_datetime('2000-01-01'), pd.to_datetime('2015-01-01') )
ax = axs[0]
ax.plot(y)
ax = axs[1]
ax.plot(y)
ax.set_yscale('log')
remove_scientific_notation_from_vertical_axis(ax)
#axis_year(ax,'%Y')
plt.show()

In [ ]:
y = np.log(x).diff().dropna()
xs = np.linspace(y.min(), y.max(), 500)
de = scipy.stats.gaussian_kde(y)
ys = de(xs)

for density in [False, True]:
    fig, axs = plt.subplots( 1, 2, figsize = (8,3), layout = 'constrained', dpi=100 )
    for ax, log_scale in zip( axs, [False, True] ):
        ax.hist( 100 * y, bins = 100, density = True, color = 'lightblue' )
        if density:
            ax.plot( 100 * xs, ys/100, color = 'tab:blue', linewidth = 3, zorder = 10)
        ax.set_xlabel("Daily logarithmic returns (%)")
        for side in ['left', 'right', 'top']: 
            ax.spines[side].set_visible(False)
        if log_scale:
            ax.set_yscale('log')
            ax.set_ylim(1e-4,1)
        else:
            ax.set_yticks([])
        ax.set_xlim(-10, 10)
        ax.axvline( 0, linewidth = 1, linestyle = ':', color = 'white' )
    plt.show()

## Discrete gradients

In [ ]:
d = data('quakes').iloc[:,:2].iloc[:,::-1]

x, y = d.values.T
xx, yy = np.mgrid[ 
    min(x) : max(x) : 500j, 
    min(y) : max(y) : 500j, 
]
positions = np.vstack([xx.ravel(), yy.ravel()])
kernel = scipy.stats.gaussian_kde( 
    np.vstack([x, y]),
    bw_method = .2,
)
f = np.reshape(kernel(positions).T, xx.shape)

fig, axs = plt.subplots( 1, 2, figsize = (8,3), layout = 'constrained', dpi = 100)
for ax in axs: 
    ax.set_xlim( min(x), max(x) )
    ax.set_ylim( min(y), max(y) )
    ax.set_xlabel( d.columns[0] )
    ax.set_ylabel( d.columns[1] )
    ax.set_aspect(1)
ax = axs[0]
ax.imshow(np.rot90(f), cmap='Blues', extent=[min(x), max(x), min(y), max(y)])

ax = axs[1]
ax.contourf(xx, yy, f, cmap='Blues')
#ax.imshow(np.rot90(f), cmap='Blues', extent=[xmin, xmax, ymin, ymax])
ax.contour(xx, yy, f, colors='k')
#ax.clabel(cset, inline=1, fontsize=10)
#ax.axes.xaxis.set_visible(False)
#ax.axes.yaxis.set_visible(False)
plt.show()

In [ ]:
#x = yf.Ticker("^GSPC").history(period="100y")['Close']
#returns = np.log(x).diff().dropna().values

from KDEpy import FFTKDE

cmap = 'Blues'
a = returns[:-1]
b = returns[1:]
a = uniformize(a)
b = uniformize(b)
xmin, xmax, ymin, ymax = 0,1, 0,1
bw = statsmodels.nonparametric.kernel_density.KDEMultivariate([a,b], 'cc').bw.mean()
k = 256
u, f = FFTKDE(bw=bw).fit( np.stack([a,b]).T )((k,k))
xx, yy = np.unique(u[:, 0]), np.unique(u[:, 1])
f = f.reshape(k,k).T

fig, axs = plt.subplots( 1, 2, figsize = (8,3.5), layout = 'constrained', dpi = 100 )
for ax in axs: 
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.set_aspect(1)

ax = axs[0]
#ax.pcolormesh( xx, yy, f )
ax.pcolormesh( xx, yy, f, cmap='viridis_r' )

ax = axs[1]
ax.contourf(xx, yy, f, cmap=cmap)
ax.contour(xx, yy, f, colors='k')
plt.show()

In [ ]:
fig, axs = plt.subplots( 9, 16, figsize = (16,9), layout = 'constrained', dpi = 50 )
for ax in axs.flatten():
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.set_aspect(1)
for ax, cmap in zip( 
    axs.flatten(), 
    #[ 'magma', 'inferno', 'plasma', 'viridis', 'cividis', 'twilight', 'twilight_shifted', 'turbo', 'Blues' ],
    matplotlib.colormaps._cmaps.keys(),
):
    ax.pcolormesh( xx, yy, f, cmap=cmap )
plt.show()

In [ ]:
scale = 1.3
fig, axs = plt.subplots( 9, 16, figsize = (scale*16,scale*9), layout = 'constrained', dpi = 50 )
for ax in axs.flatten():
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.set_aspect(1)
for ax, cmap in zip( 
    axs.flatten(), 
    #[ 'magma', 'inferno', 'plasma', 'viridis', 'cividis', 'twilight', 'twilight_shifted', 'turbo', 'Blues' ],
    matplotlib.colormaps._cmaps.keys(),
):
    ax.pcolormesh( xx, yy, f, cmap=cmap )
    ax.set_title(cmap)
plt.show()

## Thick lines

In [ ]:
x = pd.DataFrame( { k: v['Close'] for k, v in stocks.items() } )
x = x.tail(5000)
x = x.iloc[::21,:]  # Colours are harder to distinguish if the curve is smooth
x = x / np.array( [ x[u].dropna().iloc[0] for u in x.columns ] )  # Start all the time series at 1
fig, axs = plt.subplots(1,2, figsize = (12, 5), layout = 'constrained', dpi = 100 )
ax = axs[0]
x.plot(ax=ax, linewidth = .5)
ax = axs[1]
x.plot( ax=ax, linewidth = 2)
for ax in axs: 
    ax.set_yscale('log')
    axis_year( ax, "%y" )
    ax.set_ylabel("Price")
plt.show()

## Thicker lines in the legend

In [ ]:
x = pd.DataFrame( { k: v['Close'] for k, v in stocks.items() } )
x = x.tail(5000)
x = x.iloc[::21,:]
x = x / np.array( [ x[u].dropna().iloc[0] for u in x.columns ] )  # Start all the time series at 1
fig, axs = plt.subplots(1,2, figsize = (12, 5), layout = 'constrained', dpi = 100)
ax = axs[0]
x.plot(ax=ax, linewidth = .5)
ax = axs[1]
x.plot( ax=ax, linewidth = 2)
leg = ax.legend()
for i in leg.legend_handles:
    i.set_linewidth(7)
    i.set_solid_capstyle('butt')
for ax in axs: 
    ax.set_yscale('log')
    axis_year( ax, "%y" )
    ax.set_ylabel("Price")
plt.show()

## TODO Labels on the plot

## Legend order

In [ ]:
returns = np.log(x).diff().rolling(21).sum()
mu = returns.mean()
V = returns.cov()
corrplot( cov2cor(V), figsize = (3,3), order = True )

results = {}
for s in np.linspace( 0, .5, 100 ):
    n = len(mu)
    w = cp.Variable(n)
    objective = cp.sum( cp.multiply( w, mu ) )
    constraints = [ w >= 0, cp.sum(w) == 1, cp.quad_form(w,V) <= s**2 ]
    prob = cp.Problem( cp.Maximize(objective), constraints )
    if np.isfinite( prob.solve( solver = 'ECOS' ) ):
        results[100 * s ] = pd.Series( w.value, index = mu.index )
results = 100 * pd.DataFrame( results )

In [ ]:
fig, axs = plt.subplots( 1, 2, figsize = (8, 3.5), layout = 'constrained', dpi = 100 )
ax = axs[0]
stackplot( results, ax = ax, legend = True )
ax.legend()
ax = axs[1]
stackplot( results, ax = ax, legend = True )
for ax in axs: 
    ax.set_ylim(0,100)
    ax.set_xlim( np.min(results.columns), np.max(results.columns) - 1 )
    ax.set_ylabel( "Weight (%)" )
    ax.set_xlabel( "Annualized Volatility (%)" )
plt.show()

In [ ]:
scale = .4
fig, ax = plt.subplots(figsize = (scale*16,scale*9), layout = 'constrained', dpi = 100)
stackplot( results, ax = ax, legend = True )
#ax.legend()
ax.set_ylim(0,100)
#ax.set_xscale('log')
ax.set_xlim( np.min(results.columns), np.max(results.columns) - 1 )
ax.set_ylabel( "Weight (%)" )
ax.set_xlabel( "Annualized Volatility (%)" )
ax.set_title( "Optimal portfolios" )
plt.show()

## Colour palettes

In [ ]:
d = data("Electricity")
C = d.corr()
for cmap in ['hsv', 'jet', 'turbo']:  # A few non-diverging colourmaps
    fig, axs = plt.subplots(1, 2, figsize = (8,3.5), layout = 'constrained')
    ax = axs[0]
    corrplot( C, ax = ax, labels = True, order = True, vmin = C.values.min(), vmax = C.values.max(), cmap = cmap )
    ax = axs[1]
    #sns.clustermap(d.corr(), vmin=-1, vmax=+1, cmap='RdBu', figsize=(5, 5), annot=True, fmt=".2f")
    corrplot( C, ax = ax, labels = True, order = True )
    plt.show()

In [ ]:
plt.rcParams['figure.dpi'] = 100
sns.clustermap(d.corr(), vmin=-1, vmax=+1, cmap='RdBu', figsize=(5, 5), annot=True, fmt=".2f")
#for cmap in ['jet', 'hsv', 'turbo']:
cmap = 'hsv'
sns.clustermap( C, annot=True, fmt=".2f", figsize=(5,5), cmap=cmap)

In [ ]:
# Code from: https://matplotlib.org/stable/users/explain/colors/colormaps.html

plt.rcParams['figure.dpi'] = 100

from matplotlib import colormaps
from colorspacious import cspace_converter
import matplotlib as mpl

list(colormaps)
cmaps = {}

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_color_gradients(category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.22
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(6.4, figh))
    #fig, axs = plt.subplots(nrows=nrows + 1, figsize=(10, figh))
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.15 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=14)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=mpl.colormaps[name])
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

    # Save colormap list for later.
    cmaps[category] = cmap_list

plot_color_gradients('Perceptually Uniform Sequential',
                     ['viridis', 'plasma', 'inferno', 'magma', 'cividis'])

plot_color_gradients('Sequential',
                     ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                      'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                      'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'])

plot_color_gradients('Sequential (2)',
                     ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
                      'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
                      'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper'])

plot_color_gradients('Diverging',
                     ['PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu',
                      'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic'])

plot_color_gradients('Cyclic', ['twilight', 'twilight_shifted', 'hsv'])

plot_color_gradients('Qualitative',
                     ['Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2',
                      'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b',
                      'tab20c'])

plot_color_gradients('Miscellaneous',
                     ['flag', 'prism', 'ocean', 'gist_earth', 'terrain',
                      'gist_stern', 'gnuplot', 'gnuplot2', 'CMRmap',
                      'cubehelix', 'brg', 'gist_rainbow', 'rainbow', 'jet',
                      'turbo', 'nipy_spectral', 'gist_ncar'])

plt.show()

## TODO Darker, more saturated lines

## Horizontal labels

In [ ]:
d = data( "USArrests" )
d = d.sort_values("UrbanPop", ascending = False)
scale = .9
fig, axs = plt.subplots( 1, 2, figsize = (scale*16,scale*7.5), layout = 'constrained', dpi=100 )

ax = axs[0]
ax.bar(d.index, d['UrbanPop'] )
ax.tick_params(axis='x', labelrotation=90) 
ax.set_ylabel( "Urban population (%)" )
ax.set_xlim(-1,50)

d = d.iloc[::-1,:]
ax = axs[1]
ax.barh(d.index, d['UrbanPop'] )
ax.set_xlabel( "Urban population (%)" )
ax.set_ylim(-1,50)

for ax in axs: 
    for side in ['top', 'right']:
        ax.spines[side].set_visible(False)
plt.show()

## Bad plots: 3D surface (e.g., volatility)

In [ ]:
# Data from https://www.cboe.com/delayed_quotes/spx/quote_table
files = [ u for u in os.listdir( "../data" ) if u.startswith("spx_") ]
d = pd.concat( [ pd.read_csv( f"../data/{u}", skiprows = 3) for u in files ] )
d = d.drop_duplicates()
d.columns
d = d[['Expiration Date', 'IV', 'Strike', 'IV.1' ]]
d['Expiration Date'] = [ dateparser.parse(u) for u in d['Expiration Date'] ]
today = d['Expiration Date'].min()
d['Expiry'] = d['Expiration Date'] - today

In [ ]:
x = np.array( [ u.days for u in d['Expiry'] ] )
y = d['Strike']
z = d['IV']
X = np.vstack( [ x, y ] ).T
xs = np.linspace( x.min(), x.max(), 500 )
ys = np.linspace( y.min(), y.max(), 500 )
Xs = pd.merge( 
    pd.DataFrame( { 'x': xs } ),
    pd.DataFrame( { 'y': ys } ),
    how = 'cross',
).values
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
model = Pipeline( [ 
    ('rescale', MinMaxScaler()), 
    ('regression', KNeighborsRegressor(n_neighbors = 20, weights = 'distance')),
] )
model.fit(X, z)
Zs = model.predict( Xs )
Zs = np.vstack( [Xs.T, Zs] ).T
Zs = pd.DataFrame( Zs, columns = ['Expiry', 'Strike', 'IV'] )
Zs = Zs.pivot( index = 'Expiry', columns = 'Strike', values = 'IV' )

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), layout = 'constrained', dpi=100)
i = pcolormesh(np.log(.1+Zs), ax = ax, cmap='Reds')
ax.set_xlabel( Zs.columns.name )
ax.set_ylabel( Zs.index.name )
#plt.colorbar(i, ax=ax)
ax.set_title( "Implied volatility" )
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), layout = 'constrained', dpi=100)
#pcolormesh(np.log(.1+Zs), ax = ax, cmap='Reds')
ax.contourf(Zs.columns, Zs.index, np.log(.1+Zs), cmap='Reds')
ax.set_xlabel( Zs.columns.name )
ax.set_ylabel( Zs.index.name )
ax.set_title( "Implied volatility" )
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (5,4), subplot_kw={"projection": "3d"}, layout='constrained', dpi=100)
ax.plot_surface( *np.meshgrid(xs, ys), np.log(.1+Zs).T, cmap = 'turbo' )
ax.set_xlabel("Strike")
ax.set_ylabel("Expiry")
ax.set_zlabel("log(IV)")
plt.show()

In [ ]:
# Yahoo no longer provides Options data (at least, not in a usable way)
#from pandas_datareader.data import Options
#ticker = 'SPY'
#tape = Options(ticker, 'yahoo')
#data = tape.get_all_data()

## TODO Bad plots: 3D histogram

## TODO Bad plots: 3D pie chart

## Bad plots: pie chart (vs doughnut, barplot)

In [ ]:
def doughnut_plot(x, ax=None, pie = False):
    assert len(x.shape) == 1, "x should be 1-dimensional"
    assert isinstance( x, pd.Series )
    ax_was_None = ax is None
    if ax_was_None:
        fig, ax = plt.subplots()
    ax.pie(
        x,
        labels = x.index,
        #explode=.05*np.ones(len(x)),   # Bad idea: looks irregular if there are very large and very small groups
        wedgeprops = { 'linewidth' : 7, 'edgecolor' : 'white' },   # Same effect as "explode", without the problems
        # autopct='%1.1f%%', pctdistance=.85,   # Show the percentages
    )
    circle = plt.Circle(xy=(0, 0), radius=0.75, facecolor='white')
    if not pie: 
        ax.add_artist(circle)
    if ax_was_None:
        plt.show()

np.random.seed(0)
x = np.random.poisson(10, size = 5)
x = pd.Series( x, index = ['A', 'B', 'C', 'D', 'E'] )

fig, axs = plt.subplots(1, 2, figsize = (8, 3.5), layout = 'constrained', dpi=100)
doughnut_plot(x, ax=axs[0], pie=True)
doughnut_plot(x, ax=axs[1])
plt.show()
x

In [ ]:
col = cycle( plt.get_cmap("tab10").colors )
col = islice(col,len(x))
col = list(col)

for axis in [True,False]:
    fig, axs = plt.subplots(1, 2, figsize = (8, 3.5), layout = 'constrained', dpi=100)
    doughnut_plot(x, ax=axs[0], pie=True)
    ax = axs[1]
    ax.barh(x.index, x.values, color=col)
    ax.set_ylim(4.5,-.5)
    for side in ['left', 'top', 'right']: 
        ax.spines[side].set_visible(False)
    if not axis: 
        for i in range(len(x)):
            ax.text( x.iloc[i], i, f" {x.iloc[i]}", ha='left', va='center' )
        ax.spines['bottom'].set_visible(False)
        ax.set_xticks([])
    plt.show()